In [245]:
import math
import matplotlib.pyplot as plt
import ROOT
import copy
import uproot
from ROOT import TCanvas, TFile, TProfile, TNtuple, TH1F, TH2F, TF1, TLegend, gStyle, TF2, TCutG
from ROOT import gROOT, gBenchmark, gRandom, gSystem
ROOT.gStyle.SetLabelSize(0.06, "X")  # Increase X-axis label font size
ROOT.gStyle.SetLabelSize(0.06, "Y")  # Increase Y-axis label font size
ROOT.gStyle.SetTitleSize(0.06, "X")  # Increase X-axis title font size
ROOT.gStyle.SetTitleSize(0.06, "Y")  # Increase Y-axis title font size
ROOT.gStyle.SetOptTitle(1)  # Enable the title
ROOT.gStyle.SetTitleSize(0.09, "H") 

ROOT.gStyle.SetGridStyle(3)

# Set margins
ROOT.gStyle.SetPadLeftMargin(0.2)
ROOT.gStyle.SetPadRightMargin(0.2)
ROOT.gStyle.SetPadBottomMargin(0.2)

# Disable statistics box
ROOT.gStyle.SetOptStat(0)

# Set ROOT font style
ROOT.gStyle.SetLabelFont(132, "XYZ")  # Set font to Palatino for axis labels
ROOT.gStyle.SetTitleFont(132, "XYZ")  # Set font to Palatino for axis titles
ROOT.gStyle.SetTitleFont(132, "H")  # Set font to Palatino for axis titles
ROOT.gStyle.SetTextFont(132) 
colors = [ROOT.kBlue -1, ROOT.kAzure -2, ROOT.kCyan-6, ROOT.kOrange, ROOT.kOrange+7]

import numpy as np
from uits3_krakow22.src.Event import Event
from uits3_krakow22.src.Cluster import Cluster
from uits3_krakow22.src.Track import Track
from uits3_krakow22.src.Vertex import Vertex
from uits3_krakow22.src.Utils import *

from ipywidgets import IntProgress
import matplotlib.pyplot as plt
from IPython.display import display,HTML,clear_output
display(HTML("<style>.container { width:95% !important; }</style>"))
display(HTML("<style>table {float:left;}</style>"))


from skspatial.plotting import plot_3d
from skspatial.objects import Points, Cylinder, Plane

# Load Data

In [2]:
#path = "/home/cehrich/Software/uits3_krakow22/uits3_krakow22/output/yze6run456065424_221112065722.pkl" #200 MeV
#path = "/home/cehrich/Software/uits3_krakow22/uits3_krakow22/output/yze6run457063715_221113063928.pkl" #200 MeV
#path = "/home/cehrich/Software/uits3_krakow22/uits3_krakow22/output/yze6run456114514_221112120005.pkl" #80 MeV
#path = "/home/cehrich/Software/uits3_krakow22/uits3_krakow22/output/yzrun457063715_221113063928.pkl" #full file, up up
#path = "/home/cehrich/Software/uits3_krakow22/run457063715_221113063928_clusters.pkl" # full file up down
#path = '/home/cehrich/Software/uits3_krakow22/pickleNeedsFlip/yzAlign_run456195948_221112200957.pkl'
#path = '/home/cehrich/Software/uits3_krakow22/pickleNeedsFlip/updownrun456195948_221112200957.pkl'


#first weekend files

#path1 = "/home/cehrich/Software/uits3_krakow22/noFlipNeeded/run447162516_221106162754.pkl" #200 MeV,14.9Gb, threshold not mentioned Run 57
#path2 = "/home/cehrich/Software/uits3_krakow22/noFlipNeeded/run445203522_221104212842.pkl" #120 MeV, 2.3Gb, threshold not mentioned Run 031, may be junk
#path3 = "/home/cehrich/Software/uits3_krakow22/noFlipNeeded/run446172943_221105174846.pkl" #80 MeV,  2.2Gb, threshold not mentioned Run 38

#second weekend files

#100e
path1 = "/home/cehrich/Software/uits3_krakow22/pickleNeedsFlip/run456195948_221112200957.pkl" #200MeV 100e
path2 = "/home/cehrich/Software/uits3_krakow22/pickleNeedsFlip/run456092458_221112092919.pkl" #120 MeV 100e
path3 = "/home/cehrich/Software/uits3_krakow22/noFlipNeeded/run456114514_221112120005.pkl" #80MeV 100e

#200e
path4 = "/home/cehrich/Software/uits3_krakow22/pickleNeedsFlip/run461175005_221114175310.pkl" #200MeV 200e
path5 = "/home/cehrich/Software/uits3_krakow22/noFlipNeeded/run456100239_221112100811.pkl" #120MeV 200e
path6 = "/home/cehrich/Software/uits3_krakow22/noFlipNeeded/run461201255_221114201300.pkl" #80MeV 200e

#300e
#path7= "/home/cehrich/Software/uits3_krakow22/noFlipNeeded/noisyLooking/run456083237_221112083509.pkl" #200MeV 300e (junk file)
#path8 = "/home/cehrich/Software/uits3_krakow22/noFlipNeeded/noisyLooking/run456083236_221112083241.pkl" #200MeV 300e (junk file)
path9 = "/home/cehrich/Software/uits3_krakow22/noFlipNeeded/run456103757_221112104316.pkl" #120MeV 300e



events1 = []
events2 = []
events3 = []
events4 = []
events5 = []
events6 = []
events8 = []
events9 = []

allPaths=[path1, path2, path3, path4, path5, path6,path9]#,, path8, path9]
#allPaths=[path4, path5, path6]
allEvents=[events1, events2, events3, events4, events5, events6, events9]
#allPaths=[path4]#200MeV 100e

#nEvents = -1 
nEvents = 1e7


for pathNum, path in enumerate(allPaths):
    if nEvents > 0 :
        f = IntProgress(min=0, max=nEvents, description="Processing:")
        display(f)
    for i,event in enumerate(readEvents(path, nEvents=nEvents)):
        if nEvents > 0 and i>0 and i%1e4 == 0: f.value += 1e4
        EventStatus = True
        event.clusters = [cluster for cluster in event.clusters]
        if False: # pathNum==1: #only for 6 alpide files
            for alpide in ["ALPIDE_0","ALPIDE_1","ALPIDE_2","ALPIDE_3","ALPIDE_5"]: 
                clusters = event.selectDetector(alpide)
                if len(clusters) != 1:                                        #select for single cluster events that show up on all detectors only
                    EventStatus = False
                    break
            if EventStatus: 
                allEvents[pathNum].append(copy.deepcopy(event))
            
        else:
            
            for alpide in ["ALPIDE_0","ALPIDE_1","ALPIDE_2","ALPIDE_3","ALPIDE_4"]: 
                clusters = event.selectDetector(alpide)
                if len(clusters) != 1:                                        #select for single cluster events that show up on all detectors only
                    EventStatus = False
                    break
            if EventStatus: 
                allEvents[pathNum].append(copy.deepcopy(event))
    
    if 'f' in locals():
        f.bar_style = "success"
        print("In path "+str(pathNum+1)+" Found events:",len(allEvents[pathNum]),", Fraction:",len(allEvents[pathNum])*100/nEvents,"% in file "+path)

IntProgress(value=0, description='Processing:', max=10000000)

Going to process 10000000.0 events.
In path 1 Found events: 10164 , Fraction: 0.10164 % in file /home/cehrich/Software/uits3_krakow22/pickleNeedsFlip/run456195948_221112200957.pkl


IntProgress(value=0, description='Processing:', max=10000000)

Going to process 10000000.0 events.
In path 2 Found events: 11556 , Fraction: 0.11556 % in file /home/cehrich/Software/uits3_krakow22/pickleNeedsFlip/run456092458_221112092919.pkl


IntProgress(value=0, description='Processing:', max=10000000)

...

Going to process 10000000.0 events.
In path 5 Found events: 7773 , Fraction: 0.07773 % in file /home/cehrich/Software/uits3_krakow22/noFlipNeeded/run456100239_221112100811.pkl


IntProgress(value=0, description='Processing:', max=10000000)

Going to process 10000000.0 events.
In path 6 Found events: 2422 , Fraction: 0.02422 % in file /home/cehrich/Software/uits3_krakow22/noFlipNeeded/run461201255_221114201300.pkl


IntProgress(value=0, description='Processing:', max=10000000)

Going to process 10000000.0 events.
In path 7 Found events: 6352 , Fraction: 0.06352 % in file /home/cehrich/Software/uits3_krakow22/noFlipNeeded/run456103757_221112104316.pkl


# Flip

In [3]:
indexToBeFlipped = [0,1,3]
straight1=[]
straight2=[]
straight3=[]
straight4=[]
straight5=[]
straight6=[]
straight7=[]
straight8=[]
straight9=[]

straightEvents = [straight1, straight2,straight3, straight4, straight5, straight6, straight9]
for pathNum in range(len(straightEvents)):
    if pathNum in indexToBeFlipped:
        for event in allEvents[pathNum]:
            for cluster in event.clusters:
                cluster=cluster.flipYaxis()
            straightEvents[pathNum].append(event)
    else:
        straightEvents[pathNum]=allEvents[pathNum]


# Add Tracks

In [4]:
tracked1=[]
tracked2=[]
tracked3=[]
tracked4=[]
tracked5=[]
tracked6=[]
tracked7=[]
tracked8=[]
tracked9=[]

eventsTracked = [tracked1, tracked2, tracked3, tracked4, tracked5, tracked6, tracked9]

for pathNum, events in enumerate(straightEvents):
    for event in events:
        TrackLeft = Track()
        if False: #pathNum==1:
            TrackLeft.fromClusters([event.selectDetector(alpide)[0] for alpide in ["ALPIDE_3","ALPIDE_5"]])
        else:
            TrackLeft.fromClusters([event.selectDetector(alpide)[0] for alpide in ["ALPIDE_3","ALPIDE_4"]])
        
        TrackRight = Track()
        TrackRight.fromClusters([event.selectDetector(alpide)[0] for alpide in ["ALPIDE_0","ALPIDE_1","ALPIDE_2"]])
        vertex = Vertex()
        vertex.fromTracks([TrackLeft,TrackRight])
        event.addTrack(TrackLeft)
        event.addTrack(TrackRight)
        event.vertex = vertex
        event.calculateMeanClusterSize()
        eventsTracked[pathNum].append(event)
    #allEvents[pathNum]=eventsTracked[pathNum]

In [5]:
trackedAligned1=[]
trackedAligned2=[]
trackedAligned3=[]
trackedAligned4=[]
trackedAligned5=[]
trackedAligned6=[]
trackedAligned7=[]
trackedAligned8=[]
trackedAligned9=[]
A1_fixed_A1_A2_prealigned={ 
    'ALPIDE_0': [0.0, -0.0005859375, 0], 
    'ALPIDE_1': [1.104541, 0.0149414062, 0], 
    'ALPIDE_2': [0.2, 0.0, 0], 
    'ALPIDE_3': [0.0, -0.0035156249999999997, 0], 
    'ALPIDE_4': [0.0, -0.209765625, 0]
}
best_displacement_no_multiple_scat ={
    'ALPIDE_0': [0.5, -0.0546875, 0], 
    'ALPIDE_1': [0.65, -0.0546875, 0], 
    'ALPIDE_2': [0.75, -0.06875, 0], 
    'ALPIDE_3': [0.2, -0.125, 0], 
    'ALPIDE_4': [0.5546875, -0.05, 0]
}
alignment=best_displacement_no_multiple_scat
eventsTrackedAligned = [trackedAligned1, trackedAligned2, trackedAligned3, trackedAligned4, trackedAligned5, trackedAligned6, trackedAligned9]
straightEventsCopy = copy.deepcopy(straightEvents)
for pathNum, events in enumerate(straightEventsCopy):
    for event in events:
        TrackLeft = Track()
        TrackLeft.fromClusters([event.selectDetector(alpide)[0].alignLocal(alignment.get(alpide)[0:3]) for alpide in ["ALPIDE_3","ALPIDE_4"]])
        #TrackLeft.fromClusters([event.selectDetector(alpide)[0].twistTopDown(twistDisplacement[index+3]) for index, alpide in enumerate(["ALPIDE_3","ALPIDE_4"])])  #angles
    
        TrackRight = Track()
        TrackRight.fromClusters([event.selectDetector(alpide)[0].alignLocal(alignment.get(alpide)[0:3]) for alpide in ["ALPIDE_0","ALPIDE_1","ALPIDE_2"]])
        #TrackRight.fromClusters([event.selectDetector(alpide)[0].twistTopDown(twistDisplacement[index]) for index, alpide in enumerate(["ALPIDE_0","ALPIDE_1","ALPIDE_2"])]) #angles
 


        vertex = Vertex()
        vertex.fromTracks([TrackLeft,TrackRight])
        event.addTrack(TrackLeft)
        event.addTrack(TrackRight)
        event.vertex = vertex
        event.calculateMeanClusterSize()
        eventsTrackedAligned[pathNum].append(event)
    #allEvents[pathNum]=eventsTracked[pathNum]

# Plot

In [8]:
%jsroot on

#the file to be plotted
fileIndex=0

hVertexXY = ROOT.TH2F("VertexXY","VertexXY from File "+allPaths[fileIndex][-29:],300,-3.5,3.5,100,-3.5,3.5)
hVertexXY.GetXaxis().SetTitle("Vertex X position [mm]")
hVertexXY.GetYaxis().SetTitle("Vertex Y position [mm]")
hVertexXZ = ROOT.TH2F("VertexXZ","VertexXZ from File "+allPaths[fileIndex][-29:],300,-3.5, 3.5,100,-23,-16)
hVertexXZ.GetXaxis().SetTitle("Vertex X position [mm]")
hVertexXZ.GetYaxis().SetTitle("Vertex Z position [mm]")
hVertexYZ = ROOT.TH2F("VertexYZ","VertexYZ from File "+allPaths[fileIndex][-29:],300,-30,30,100,-30,30)
hVertexYZ.GetXaxis().SetTitle("Vertex Y position [mm]")
hVertexYZ.GetYaxis().SetTitle("Vertex Z position [mm]")


#first or second weekend because of target shift
weekend=2
# if fileIndex<3:
#     weekend=1
# else:
#     weekend=2
if (weekend==1):
    xMin=-1
    xMax=1.5

    yMin=-4.25
    yMax=3.75

    zMin=-19.5#75
    zMax=-19.4#25
    

elif (weekend==2):
    xMin=-1
    xMax=1.5

    yMin=-4
    yMax=4

    zMin=-18.5
    zMax=-18

    
volume= (xMax-xMin) * (yMax-yMin) * (zMax-zMin)
 
        
for event in eventsTracked[fileIndex]:
    #event.calculateMeanClusterSize()
   # if event.meanClusterSize > 30:
    #for i in range(round(event.meanClusterSize)):
    #if( xMin < event.vertex.point[0] < xMax and yMin < event.vertex.point[1] < yMax and zMin < event.vertex.point[2] < zMax):
    #if( zMin < event.vertex.point[2] < zMax):
    hVertexXY.Fill(event.vertex.point[0],event.vertex.point[1])
    hVertexXZ.Fill(event.vertex.point[0],event.vertex.point[2])
    hVertexYZ.Fill(event.vertex.point[1],event.vertex.point[2])
             
        
        
        
canvas = ROOT.TCanvas("","",1300,500)
canvas.Divide(3)

canvas.cd(1)
ROOT.gPad.SetGrid(2)
hVertexXY.Draw("COLZ") #surf2

canvas.cd(2)
ROOT.gPad.SetGrid(2)
hVertexXZ.Draw("COLZ")

canvas.cd(3)
ROOT.gPad.SetGrid(2)
hVertexYZ.Draw("COLZ")

canvas.Draw()

In [8]:
%jsroot on

energy_and_thresholds = ["200 MeV, 100e", "120 MeV, 100e", "80 MeV, 100e", "200 MeV, 200e", "120 MeV, 200e", "80 MeV, 200e", "120 MeV, 300e"]
#hVertexXYs = [ROOT.TH2F("VertexXY","VertexXZ from File "+allPaths[i][-29:],200,-10,10,200,-10,10) for i in range(6)]
hVertexXYs = [ROOT.TH2F("VertexXY","VertexXZ from File "+energy_and_thresholds[i],100,-2,2,100,-21,-17) for i in range(6)]#[-29:]

for hVertexXY in hVertexXYs:
    hVertexXY.GetXaxis().SetTitle("Vertex X position [mm]")
    hVertexXY.GetYaxis().SetTitle("Vertex Y position [mm]")
#the file to be plotted
for fileIndex in range(6): 

    #first or second weekend because of target shift
    weekend=2
    # if fileIndex<3:
    #     weekend=1
    # else:
    #     weekend=2
    if (weekend==1):
        xMin=-1
        xMax=1.5

        yMin=-4.25
        yMax=3.75

        zMin=-19.5#75
        zMax=-19.4#25


    elif (weekend==2):
        xMin=-1
        xMax=1.5

        yMin=-4
        yMax=4

        zMin=-18.5
        zMax=-18


    volume= (xMax-xMin) * (yMax-yMin) * (zMax-zMin)


    for event in eventsTrackedAligned[fileIndex]:
        #event.calculateMeanClusterSize()
       # if event.meanClusterSize > 30:
        #for i in range(round(event.meanClusterSize)):
        #if( xMin < event.vertex.point[0] < xMax and yMin < event.vertex.point[1] < yMax and zMin < event.vertex.point[2] < zMax):
        #if( zMin < event.vertex.point[2] < zMax):
        #if event.vertex.dca < 2:
        hVertexXYs[fileIndex].Fill(event.vertex.point[0],event.vertex.point[2])
             
        
        
        
canvas = ROOT.TCanvas("","",1200,900)
canvas.Divide(3,2)
for i in range(6):
    canvas.cd(i+1)
    ROOT.gPad.SetGrid(2)
    hVertexXYs[i].Draw("COLZ") #surf2

canvas.Draw()    
        
canvas.SaveAs("VertexXZs_Best_Align.pdf")

Warning in <TROOT::Append>: Replacing existing TH1: VertexXY (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: VertexXY (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: VertexXY (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: VertexXY (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: VertexXY (Potential memory leak).
Info in <TCanvas::Print>: pdf file VertexXZs_Best_Align.pdf has been created


In [10]:
print(len(eventsTracked))

7


In [11]:
%jsroot on

#somehow find ratio between the two weekends
#if entries overlap (both nonzero), then divide 
#BUT what are we doing the ratio between?

rangNum=6
hVertexXYs = [ROOT.TH2F("VertexXY","VertexXY from File "+allPaths[i][-29:],200,-10,10,200,-10,10) for i in range(rangNum)]

for hVertexXY in hVertexXYs:
    hVertexXY.GetXaxis().SetTitle("Vertex X position [mm]")
    hVertexXY.GetYaxis().SetTitle("Vertex Y position [mm]")
#the file to be plotted
for fileIndex in range(rangNum): 

    #first or second weekend because of target shift
    weekend=0
    if fileIndex<3:
        weekend=1
    else:
        weekend=2
    if (weekend==1):
        xMin=-1
        xMax=1.5

        yMin=-4.25
        yMax=3.75

        zMin=-19.5#75
        zMax=-19.4#25


    elif (weekend==2):
        xMin=-1
        xMax=1.5

        yMin=-4
        yMax=4

        zMin=-18.5
        zMax=-18


    volume= (xMax-xMin) * (yMax-yMin) * (zMax-zMin)


    for event in eventsTracked[fileIndex]:
        #event.calculateMeanClusterSize()
       # if event.meanClusterSize > 30:
        #for i in range(round(event.meanClusterSize)):
        #if( xMin < event.vertex.point[0] < xMax and yMin < event.vertex.point[1] < yMax and zMin < event.vertex.point[2] < zMax):
        #if( zMin < event.vertex.point[2] < zMax):
        #if event.vertex.dca < 2:
        hVertexXYs[fileIndex].Fill(event.vertex.point[0],event.vertex.point[1])
             
        
        
        
canvas = ROOT.TCanvas("","",1200,900)
canvas.Divide(3,2)
for i in range(rangNum):
    canvas.cd(i+1)
    ROOT.gPad.SetGrid(2)
    hVertexXYs[i].Draw("COLZ") #surf2

canvas.Draw()    
        


Warning in <TROOT::Append>: Replacing existing TH1: VertexXY (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: VertexXY (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: VertexXY (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: VertexXY (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: VertexXY (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: VertexXY (Potential memory leak).


## DCA

In [232]:
%jsroot on

detectors = ["ALPIDE_0","ALPIDE_1","ALPIDE_2","ALPIDE_3","ALPIDE_4"]
canvas = ROOT.TCanvas("","",600,450)
legend = ROOT.TLegend(0.8,0.8)
axis_start=0
axis_finish=1000
bins=int(80)
xTMin=-2
xTMax=2
yTMin=-5
yTMax=5
zTMin=-22 #bigger to accomodate targets within shift
zTMax=-17
tVolume= (xTMax-xTMin) * (yTMax-yTMin) * (zTMax-zTMin)

# Calculate histograms
hOpenningAngle  = ROOT.TH1F("DCA", "Target Region", bins, axis_start,axis_finish)
hOpenningAngle2 = ROOT.TH1F("DCA", "Target Region", bins, axis_start,axis_finish)
hOpenningAngle3 = ROOT.TH1F("DCA", "Target Region", bins, axis_start,axis_finish)

histograms = [hOpenningAngle, hOpenningAngle2, hOpenningAngle3]

for index, eventSet in enumerate(eventsTrackedAligned):
    if index < 3:
        for event in eventSet: 
            if 85 < event.vertex.openingAngle*360/(2*math.pi) < 90 and abs(event.vertex.dca2origin - 18.75) < 2 and event.vertex.dca < 2 and all([7<cluster.size<20 for cluster in event.clusters]): #2sigma of 80MeV 100e target sizes
              if(all([(-8 < cluster.localPos[0] < 8 and -3 < cluster.localPos[1] < 3) for cluster in event.clusters])): #cuts out multiple scattering with plastic window
                   if( xTMin < event.vertex.point[0] < xTMax and yTMin < event.vertex.point[1] < yTMax and zTMin < event.vertex.point[2] < zTMax):
                        histograms[index].Fill(event.vertex.dca*1000)

#Draw histograms
#colors = [1,2,3,4,6,7,8,9]
labels=["200 MeV, 100e", "120 MeV, 100e", " 80 MeV, 100e"]
gauss1 = ROOT.TF1("gauss", "gaus")
gauss2 = ROOT.TF1("gauss", "gaus")
gauss3 = ROOT.TF1("gauss", "gaus")

gauss= [gauss1,gauss2, gauss3]
lines = []
ppOpAng=[87.08, 88.2, 88.76]
p2pOpAng=[82.56, 79.82, 74.78]

ROOT.gStyle.SetPadRightMargin(0.21)
ROOT.gPad.SetGrid(2)
for index,dcahist in enumerate(histograms):
    if index < 3:
        dcahist.Scale(1/dcahist.Integral())
        dcahist.SetLineColor(colors[index+2])
        dcahist.SetMarkerColor(colors[index+2])
        #dcahist.SetMarkerStyle(1)
        dcahist.SetLineWidth(3)
        dcahist.SetYTitle("Counts / Normalized to Integral")
        #dcahist.SetLineStyle(2)
        dcahist.SetXTitle("DCA [#mum]")
        dcahist.GetXaxis().SetNdivisions(5,5,0)
        dcahist.Draw("SAME HIST")
        legend.AddEntry(dcahist, labels[index])
ROOT.gStyle.SetOptStat(0)
legend.Draw()
canvas.Draw()
canvas.SaveAs("DCA_All.pdf")

Warning in <TROOT::Append>: Replacing existing TH1: DCA (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: DCA (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: DCA (Potential memory leak).
Warning in <TLegend::Paint>: Legend too large to be automatically placed; a default position is used
Info in <TCanvas::Print>: pdf file DCA_All.pdf has been created


## Openning Angle

In [165]:
%jsroot on

detectors = ["ALPIDE_0","ALPIDE_1","ALPIDE_2","ALPIDE_3","ALPIDE_4"]
canvas = ROOT.TCanvas("","", 696, 472) #,2400,900) #1200 looks nicer but 2400 matches tobis plots
#canvas = ROOT.TCanvas("","",1200,450)
#canvas.Divide(3,1)

axis_start=85.5 #72.5
axis_finish=91.5
bins=100
xTMin=-2
xTMax=2

yTMin=-5
yTMax=5

zTMin=-22 #bigger to accomodate targets within shift
zTMax=-17

tVolume= (xTMax-xTMin) * (yTMax-yTMin) * (zTMax-zTMin)

xAMin=4.5
xAMax=8

yAMin=-4
yAMax=4

zAMin=-20
zAMax=-17.5

aVolume= (xAMax-xAMin) * (yAMax-yMin) * (zAMax-zAMin)

xMin=-1
xMax=1.5

yMin=-4
yMax=4

zMin=-18.5
zMax=-18

volume= (xMax-xMin) * (yMax-yMin) * (zMax-zMin)

# Calculate histogram

hOpenningAngle  = ROOT.TH1F("Openning Angle", "Target Region 200 MeV, 100e", bins, axis_start,axis_finish)
hOpenningAngle2 = ROOT.TH1F("Openning Angle", "Target Region 120 MeV, 100e", bins, axis_start,axis_finish)
hOpenningAngle3 = ROOT.TH1F("Openning Angle", "Target Region 80  MeV, 100e", bins, axis_start,axis_finish)
hOpenningAngle4 = ROOT.TH1F("Openning Angle", "Target Region 200 MeV, 100e",bins, axis_start,axis_finish)
hOpenningAngle5 = ROOT.TH1F("Openning Angle", "Target Region 120 MeV, 100e",bins, axis_start,axis_finish) 
hOpenningAngle6 = ROOT.TH1F("Openning Angle", "Target Region 80  MeV, 100e", bins, axis_start,axis_finish)

eLabel=["200 MeV", "120 MeV", "80 MeV"]

hDivHists= [ROOT.TH1F("Openning Angle Divided", "Opening Angle First Week Divided By Second Week "+eLabel[i], 30,60,120) for i in range(3)]

for hDivHist in hDivHists:
    hDivHist.GetXaxis().SetTitle("Openning Angle [^{o}]")

histograms = [hOpenningAngle, hOpenningAngle2, hOpenningAngle3,hOpenningAngle4, hOpenningAngle5, hOpenningAngle6]


for index, eventSet in enumerate(eventsTrackedAligned):
    if index < 6:
        for event in eventSet: 
            #if( xAMin < event.vertex.point[0] < xAMax and yAMin < event.vertex.point[1] < yAMax and zAMin < event.vertex.point[2] < zAMax):
            histograms[1].Fill(event.vertex.openingAngle*360/(2*math.pi),1/tVolume)
            if( xTMin < event.vertex.point[0] < xTMax and yTMin < event.vertex.point[1] < yTMax and zTMin < event.vertex.point[2] < zTMax):
                histograms[index].Fill(event.vertex.openingAngle*360/(2*math.pi))

               
                
#Draw histogram
colors = [1,2,3,4,6,7,8,9]
labels=["200MeV, 100e", "120MeV, 100e", "80MeV, 100e","200MeV, 100e", "120MeV, 100e", "80MeV, 100e"]
gauss1 = ROOT.TF1("gauss", "gaus")
gauss2 = ROOT.TF1("gauss", "gaus")
gauss3 = ROOT.TF1("gauss", "gaus")
gauss4 = ROOT.TF1("gauss", "gaus")
gauss5 = ROOT.TF1("gauss", "gaus")
gauss6 = ROOT.TF1("gauss", "gaus")




gauss= [gauss1,gauss2, gauss3,gauss4,gauss5, gauss6]
lines = []
#200MeV, 120MeV, 80MeV

ppOpAng=[87.08, 88.2, 88.76]
p2pOpAng=[82.56, 79.82, 74.78]


for row in range(1):
    for index,chist in enumerate(histograms[0:3]):
    #for index,chist in enumerate(hDivHists):
        #canvas.cd(index+1)
        legend = ROOT.TLegend(0.8,0.8)

        #chist.Divide(histograms[index],histograms[index+3])
        #print(chist)
        #gauss[index].SetLineColor(colors[index])


        #chist.SetBinErrorOption(ROOT.TH1.kNoError)
        #chist.Fit(gauss[index])

        # retrieve the fit parameters
        #mean = gauss[index].GetParameter(1)
        #sigma = gauss[index].GetParameter(2)
        #sigNormd = sigma/np.sqrt(chist.GetEntries())

        #chist.SetLineColor(colors[index])
        #chist.SetMarkerColor(colors[index])    
        chist.SetXTitle("Opening Angle / ^{o} ")
        chist.SetYTitle("#splitline{Cluster Counts /}{Normalized to Area}") 
        #legend.AddEntry(chist,labels[index]+ " #mu= " + str(mean)[0:5]) + " and #sigma/#sqrtN= " + str(sigNormd)[0:5])
        legend.AddEntry(chist, labels[index]+ ", #mu= " + str(chist.GetMean())[0:5])    
        chist.SetMarkerStyle(20)   
        chist.Scale(1/chist.Integral())
        chist.SetMarkerStyle(20)
        #hist.SetMaximum(0.2)
        ROOT.gPad.SetGrid(2)
        #chist.SetLineColor(colors[index])
        chist.SetLineWidth(2)
        chist.Draw("SAME HIST")#"P"

        #line1 = ROOT.TLine(p2pOpAng[index], 0, p2pOpAng[index], chist.GetMaximum())
        #line1.SetLineColor(5)
        #line1.SetLineWidth(2)
        #line1.Draw()
        #lines.append(line1)
        x_max = chist.GetXaxis().GetBinCenter(chist.GetMaximumBin())
        print("X position of maximum value:", x_max)
        textAngle = ROOT.TLatex(85, 0.5, "Maximum value "+str(x_max))
        textAngle.SetTextSize(0.12)
        textAngle.SetTextColor(ROOT.kBlack)
        textAngle.Draw()


        line2 = ROOT.TLine(ppOpAng[index], 0, ppOpAng[index], chist.GetMaximum())
        line2.SetLineColor(ROOT.kGreen)
        line2.SetLineWidth(3)
        line2.SetLineStyle(2)
        line2.Draw()
        lines.append(line2)
        #pad.Update()

    # Set the legend header to the top right corner
        legend.SetHeader("Legend", "C")
        legend.SetX1NDC(0.7)
        legend.SetY1NDC(0.7)
        legend.SetX2NDC(0.9)
        legend.SetY2NDC(0.9)
        legend.Draw()

canvas.Draw()
#canvas.SetGridx(2)
#canvas.SaveAs("OpenningAngleExperimentalAll.pdf")

X position of maximum value: 86.67
X position of maximum value: 87.93
X position of maximum value: 88.35


Warning in <TROOT::Append>: Replacing existing TH1: Openning Angle (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: Openning Angle (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: Openning Angle (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: Openning Angle (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: Openning Angle (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: Openning Angle (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: Openning Angle Divided (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: Openning Angle Divided (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: Openning Angle Divided (Potential memory leak).
Warning in <TLegend::Paint>: Legend too large to be automatically placed; a default position is used


In [166]:
legends = [ROOT.TLegend(0.89,0.8) for i in range(3)] #ROOT.TLegend(histXVals[index],0.8)
for row in range(1):
    for index,chist in enumerate(histograms[0:3]):
        canvas = ROOT.TCanvas("","", 696, 472)
        chist.Draw("SAME HIST")
        legend = legends[index]
        legend.SetX1(0.65) # move legend to the right
        legend.SetX2(1.0) # move legend to the right
        legend.SetY1(0.7) # move legend to the top
        legend.SetY2(0.9) # move legend to the top
        #canvas.cd(index+1)

        # Plot the existing histogram
        
        
        # Plot the data from the file

        # fill the histogram with the values from the branch array
        # Open the file
        file = uproot.open("/home/cehrich/Software/event_generator/" +energy[index]+ "quasi.root")

        # Get the data from the file
        data = file["tree;1"]

        branch = data["Opang_degr"]


        data_hist = data_hists[index] 
        for d in branch.array():
            data_hist.Fill(d)
        data_hist.SetLineColor(2)
        data_hist.Scale(1/data_hist.Integral())
        data_hist.Draw("SAME HIST")
        legend.AddEntry(data_hist, "p-p Elastic Simulated Data, ")
        legend.AddEntry(0, str(data_hist.GetXaxis().GetBinCenter(data_hist.GetMaximumBin()))[0:5]+"^{o} Peak, and " + str(data_hist.GetMean())[0:5] +"^{o} Mean", "")
        
        if False:
            file = uproot.open("/home/cehrich/Software/event_generator/" +energy[index]+ "12Cquasi.root")

            # Get the data from the file
            Cdata = file["tree;1"]

            Cbranch = Cdata["Opang_degr"]

            Cdata_hist = Cdata_hists[index] 
            for d in Cbranch.array():
                Cdata_hist.Fill(d)
            Cdata_hist.SetLineColor(4)
            Cdata_hist.Scale(1/Cdata_hist.Integral())
            Cdata_hist.Draw("SAME HIST")
            legend.AddEntry(Cdata_hist, "^{12}C Quasi Free Simulated Data, ")
            legend.AddEntry(0, str(Cdata_hist.GetXaxis().GetBinCenter(Cdata_hist.GetMaximumBin()))[0:5]+"^{o} Peak, and " + str(Cdata_hist.GetMean())[0:5] +"^{o} Mean", "")
        #chist.Divide(histograms[index],histograms[index+3])
        #print(chist)
        #gauss[index].SetLineColor(colors[index])


        #chist.SetBinErrorOption(ROOT.TH1.kNoError)
        #chist.Fit(gauss[index])

        # retrieve the fit parameters
        #mean = gauss[index].GetParameter(1)
        #sigma = gauss[index].GetParameter(2)
        #sigNormd = sigma/np.sqrt(chist.GetEntries())

        #chist.SetLineColor(colors[index])
        #chist.SetMarkerColor(colors[index])    
        chist.SetXTitle("Opening Angle / ^{o} ")
        chist.SetYTitle("#splitline{Cluster Counts /}{Normalized to Integral}") 
        #legend.AddEntry(chist,labels[index]+ " #mu= " + str(mean)[0:5]) + " and #sigma/#sqrtN= " + str(sigNormd)[0:5])
        #legend.AddEntry(chist, labels[index]+ ", #mu= " + str(chist.GetMean())[0:5])    
        #chist.SetMarkerStyle(0)
        chist.SetLineColor(ROOT.kBlack)
        #chist.SetBinErrorOption(ROOT.TH1.kNoError)
        chist.Scale(1/chist.Integral())
        legend.AddEntry(chist, "Experimental Data, ", "l")
        legend.AddEntry(0, str(chist.GetXaxis().GetBinCenter(chist.GetMaximumBin()))[0:5]+"^{o} Peak, and " + str(chist.GetMean())[0:5]+ "^{o} Mean", "")
        chist.SetMarkerStyle(0)
        #hist.SetMaximum(0.2)
        ROOT.gPad.SetGrid(2)
        #chist.SetLineColor(colors[index])
        #chist.SetLineWidth(2)

        #line1 = ROOT.TLine(p2pOpAng[index], 0, p2pOpAng[index], chist.GetMaximum())
        #line1.SetLineColor(5)
        #line1.SetLineWidth(2)
        #line1.Draw()
        #lines.append(line1)
        x_max = chist.GetXaxis().GetBinCenter(chist.GetMaximumBin())
        print("X position of maximum value:", x_max)
        textAngle = ROOT.TLatex(85, 0.5, "Maximum value "+str(x_max))
        textAngle.SetTextSize(0.12)
        textAngle.SetTextColor(ROOT.kBlack)
        textAngle.Draw()

        chist.SetMaximum(maxYVals[index])#0.11)
        chist.SetMinimum(0)
        line2 = ROOT.TLine(ppOpAng[index], 0, ppOpAng[index], 0.072)#chist.GetMaximum())
        line2.SetLineColor(ROOT.kGreen)
        line2.SetLineWidth(3)
        line2.SetLineStyle(2)
        #line2.Draw()
        lines.append(line2)
        # ...
        # ...
            #pad.Update()
        legend.SetX1NDC(0.15)
        legend.SetY1NDC(0.15)
        legend.SetTextSize(0.03)
        canvas.Update()
        legend.Draw()
        ROOT.gStyle.SetPadLeftMargin(0.2)
        canvas.Draw()
        canvas.SaveAs("AlignedOpeningAngle"+energy[index]+".pdf")
ROOT.gStyle.SetOptStat(0)
#legend.Draw()
#legend.SetX1NDC(0.08)
#legend.SetY1NDC(0.08)
canvas.Update()
#canvas.SaveAs("AlignedOpeningAngleComparison.pdf")


X position of maximum value: 86.67
X position of maximum value: 87.93
X position of maximum value: 88.35


Info in <TCanvas::Print>: pdf file AlignedOpeningAngle200MeV.pdf has been created
Info in <TCanvas::Print>: pdf file AlignedOpeningAngle120MeV.pdf has been created
Info in <TCanvas::Print>: pdf file AlignedOpeningAngle80MeV.pdf has been created


In [151]:
%jsroot on

detectors = ["ALPIDE_0","ALPIDE_1","ALPIDE_2","ALPIDE_3","ALPIDE_4"]
canvas = ROOT.TCanvas("","", 696, 472) #,2400,900) #1200 looks nicer but 2400 matches tobis plots
#canvas.Divide(2,1)
legend = ROOT.TLegend(1.0,0.8)

xTMin=-2
xTMax=2

yTMin=-5
yTMax=5

zTMin=-22 #bigger to accomodate targets within shift

# Open the file
file = uproot.open("/home/cehrich/Software/event_generator/quasi.root")

# Get the data from the file
data = file["tree;1"]

branch = data["Opang_degr"]



for row in range(1):
    for index,chist in enumerate(histograms[0:1]):
        
    #for index,chist in enumerate(hDivHists):
        canRow=0
        if row==1:
            canRow+=3
        
        pad=canvas.cd(index+1+canRow)
        
        if row==1:
            pad.SetLogy()
        
        # Plot the existing histogram
        chist.Draw()
        
        # Plot the data from the file

        # fill the histogram with the values from the branch array

        data_hist = ROOT.TH1F("data_hist", "Data from quasi.root", bins, axis_start, axis_finish)
        for d in branch.array():
            data_hist.Fill(d)
        data_hist.SetLineColor(2)
        data_hist.Scale(1/data_hist.Integral())
        data_hist.Draw("SAME HIST")
        legend.AddEntry(data_hist, "Simulated Data, "+str(data_hist.GetXaxis().GetBinCenter(data_hist.GetMaximumBin()))[0:5]+"^{o} Peak")
        
        #chist.Divide(histograms[index],histograms[index+3])
        #print(chist)
        #gauss[index].SetLineColor(colors[index])


        #chist.SetBinErrorOption(ROOT.TH1.kNoError)
        #chist.Fit(gauss[index])

        # retrieve the fit parameters
        #mean = gauss[index].GetParameter(1)
        #sigma = gauss[index].GetParameter(2)
        #sigNormd = sigma/np.sqrt(chist.GetEntries())

        #chist.SetLineColor(colors[index])
        #chist.SetMarkerColor(colors[index])    
        chist.SetXTitle("Opening Angle / ^{o} ")
        chist.SetYTitle("#splitline{Cluster Counts /}{Normalized to Area}")
        #chist.SetBinErrorOption(ROOT.TH1.kNoError)
        #legend.AddEntry(chist,labels[index]+ " #mu= " + str(mean)[0:5]) + " and #sigma/#sqrtN= " + str(sigNormd)[0:5])
        #legend.AddEntry(chist, labels[index]+ ", #mu= " + str(chist.GetMean())[0:5])    
        #chist.SetMarkerStyle(0)
        chist.SetLineColor(ROOT.kBlack)
        chist.Scale(1/chist.Integral())

        chist.Draw("SAME HIST")#"P"
        legend.AddEntry(chist, "Experimental Data, "+str(chist.GetXaxis().GetBinCenter(chist.GetMaximumBin()))[0:5]+"^{o} Peak")
        #chist.SetMarkerStyle(20)
        #hist.SetMaximum(0.2)
        ROOT.gPad.SetGrid(2)
        #chist.SetLineColor(colors[index])
        #chist.SetLineWidth(2)

        #line1 = ROOT.TLine(p2pOpAng[index], 0, p2pOpAng[index], chist.GetMaximum())
        #line1.SetLineColor(5)
        #line1.SetLineWidth(2)
        #line1.Draw()
        #lines.append(line1)
        x_max = chist.GetXaxis().GetBinCenter(chist.GetMaximumBin())
        print("X position of maximum value:", x_max)
        textAngle = ROOT.TLatex(85, 0.5, "Maximum value "+str(x_max))
        textAngle.SetTextSize(0.12)
        textAngle.SetTextColor(ROOT.kBlack)
        textAngle.Draw()


        line2 = ROOT.TLine(ppOpAng[index], 0, ppOpAng[index], 0.072)#chist.GetMaximum())
        line2.SetLineColor(ROOT.kGreen)
        line2.SetLineWidth(3)
        line2.SetLineStyle(2)
        #line2.Draw()
        lines.append(line2)
        pad.Update()
legend.Draw()
legend.SetX1NDC(0.08)
legend.SetY1NDC(0.08)
canvas.Update()
canvas.Draw()
#canvas.SaveAs("OpeningAngleComparison.pdf")


FileNotFoundError: file not found

    '/home/cehrich/Software/event_generator/quasi.root'

Files may be specified as:
   * str/bytes: relative or absolute filesystem path or URL, without any colons
         other than Windows drive letter or URL schema.
         Examples: "rel/file.root", "C:\abs\file.root", "http://where/what.root"
   * str/bytes: same with an object-within-ROOT path, separated by a colon.
         Example: "rel/file.root:tdirectory/ttree"
   * pathlib.Path: always interpreted as a filesystem path or URL only (no
         object-within-ROOT path), regardless of whether there are any colons.
         Examples: Path("rel:/file.root"), Path("/abs/path:stuff.root")

Functions that accept many files (uproot.iterate, etc.) also allow:
   * glob syntax in str/bytes and pathlib.Path.
         Examples: Path("rel/*.root"), "/abs/*.root:tdirectory/ttree"
   * dict: keys are filesystem paths, values are objects-within-ROOT paths.
         Example: {"/data_v1/*.root": "ttree_v1"

In [295]:
%jsroot on

canvas = ROOT.TCanvas("","",1400,480)
canvas.Divide(3)
legend = ROOT.TLegend()

# Select alignment
ALPIDES = ['ALPIDE_0','ALPIDE_1', 'ALPIDE_2', 'ALPIDE_3', 'ALPIDE_4'] 
null_displacement={
     'ALPIDE_0': [0, 0, 0], 
     'ALPIDE_1': [0, 0, 0], 
     'ALPIDE_2': [0, 0, 0], 
     'ALPIDE_3': [0, 0, 0], 
     'ALPIDE_4': [0, 0, 0]
}
custom_displacement={
     'ALPIDE_0': [0, 0, 0], 
     'ALPIDE_1': [1.1, 0, 0], 
     'ALPIDE_2': [0.2, 0, 0], 
     'ALPIDE_3': [0, 0, 0], 
     'ALPIDE_4': [0, 0, 0]
}

alignment01092023={
     'ALPIDE_0': [-0.28125, -0.225, 0.15], 
     'ALPIDE_1': [-0.225, -0.24375, 0.15], 
     'ALPIDE_2': [0.3, -0.24375, 0.075], 
     'ALPIDE_3': [-0.3, -0.2625, 0.075], 
     'ALPIDE_4': [-0.3, -0.24375, 0.15]
}

best_displacement_no_multiple_scat ={
    'ALPIDE_0': [0.5, -0.0546875, 0], 
    'ALPIDE_1': [0.65, -0.0546875, 0], 
    'ALPIDE_2': [0.75, -0.06875, 0], 
    'ALPIDE_3': [0.2, -0.125, 0], 
    'ALPIDE_4': [0.5546875, -0.05, 0]
}

berkin_ml_displacement={
    'ALPIDE_0': [-0.20006977021694183, -0.8966014385223389,0], 
    'ALPIDE_1': [0.7433865666389465, -0.8977992534637451,0], 
    'ALPIDE_2': [-0.8155627250671387, -0.9625105261802673,0], 
    'ALPIDE_3': [0.10266361385583878, 0.31308597326278687,0], 
    'ALPIDE_4': [0.4219255745410919, 1.322750449180603,0]
}
temp_x_align={
    'ALPIDE_0': [0, -1.75, 0], 
    'ALPIDE_1': [1.1, 1.296875, 0], 
    'ALPIDE_2': [0.2, 1.0, 0], 
    'ALPIDE_3': [0, 1.75, 0], 
    'ALPIDE_4': [0, -1.25, 0]
}
dca_A1A2prealigned_with_completeDCA_displacement_fixed_A0const={
    'ALPIDE_0': [0, 0, 0], 
    'ALPIDE_1': [1.1046875, 0.014648437499999998, 0], 
    'ALPIDE_2': [0.19062500000000002, 0.0046875, 0],
    'ALPIDE_3': [0.009375, -0.0046875, 0], 
    'ALPIDE_4': [0.0046875, 0.0087890625, 0]
}
displacement=dca_A1A2prealigned_with_completeDCA_displacement_fixed_A0const
# Calculate histogram

# hOpenningAngle  = ROOT.TH2F("Opening Angle", "Target Region 200 MeV", 100, -8, 8, 200,80, 95)
# hOpenningAngle2 = ROOT.TH2F("Opening Angle", "Target Region 120 MeV", 100,-8,8, 200,  80, 95) 
# hOpenningAngle3 = ROOT.TH2F("Opening Angle", "Target Region 80 MeV", 100,-8,8, 200,   80, 95)
hOpenningAngle  = ROOT.TH1F("Opening Angle", "Target Region 200 MeV", 200,80, 95)
hOpenningAngle2 = ROOT.TH1F("Opening Angle", "Target Region 120 MeV", 200,  80, 95) 
hOpenningAngle3 = ROOT.TH1F("Opening Angle", "Target Region 80 MeV", 200,   80, 95)

eLabel=["200 MeV", "120 MeV", "80 MeV"]

hDivHists= [ROOT.TH1F("Openning Angle Divided", "Opening Angle First Week Divided By Second Week "+eLabel[i], 30,80,95) for i in range(3)]

for hDivHist in hDivHists:
    hDivHist.GetXaxis().SetTitle("Openning Angle [^{o}]")

histograms = [hOpenningAngle, hOpenningAngle2, hOpenningAngle3] #,hOpenningAngle4, hOpenningAngle5, hOpenningAngle6]

dcaVals = []
#trackRMSVals = []
tempEvents=copy.deepcopy(eventsTracked) #problem is that the events are changed in the algorithm so a copy must be used loadEvents("/home/cehrich/Software/uits3_krakow22/cleanEvents_brute.pkl")

#for set in tempEvents:
    #print("set", set)
for event in tempEvents[0]: #set:    
    #print("event.selectDetector(alpide)[0]", event.selectDetector(alpide)[0])
    TrackLeft = Track()
    TrackLeft.fromClusters([event.selectDetector(alpide)[0].alignLocal(displacement.get(alpide)[0:3]) for alpide in ['ALPIDE_3','ALPIDE_4']])
    #why does this line give me a list index out of range error? 
    
    TrackRight = Track()
    TrackRight.fromClusters([event.selectDetector(alpide)[0].alignLocal(displacement.get(alpide)[0:3]) for alpide in ['ALPIDE_0','ALPIDE_1','ALPIDE_2']]) 
    
    vertex = Vertex()
    vertex.fromTracks([TrackLeft,TrackRight])
    for alpide in ALPIDES:
        event.selectDetector(alpide)[0].alignLocal(displacement.get(alpide)[0:5])
    event.addTrack(TrackLeft)
    event.addTrack(TrackRight)
    event.vertex = vertex
    dcaVals.append(vertex.dca)
    #trackRMSVals.append(event.trackRMS)
 
for index, eventSet in enumerate(tempEvents):
    if index < 3:
        for event in eventSet: 
            if( xTMin < event.vertex.point[0] < xTMax and yTMin < event.vertex.point[1] < yTMax and zTMin < event.vertex.point[2] < zTMax):
                #histograms[index].Fill(event.selectDetector("ALPIDE_1")[0].localPos[1], event.vertex.openingAngle*360/(2*math.pi)) 
                histograms[index].Fill(event.vertex.openingAngle*360/(2*math.pi)) 

                
#Draw histogram
colors = [1,2,3,4,6,7,8,9]
labels=["200MeV Week 1", "120MeV Week 1", "80MeV Week 1","200MeV Week 2", "120MeV Week 2", "80MeV Week 2"]

lines = []
#200MeV, 120MeV, 80MeV

ppOpAng=[87.08, 88.2, 88.76]
p2pOpAng=[82.56, 79.82, 74.78]



xTMin=-2
xTMax=2

zTMin=-22
zTMax=-17



for row in range(1):
    for index,chist in enumerate(histograms[0:3]):
        canRow=0
        if row==1:
            canRow+=3
        
        pad=canvas.cd(index+1+canRow)
        
        if row==1:
            pad.SetLogy()
        

        #chist.SetLineColor(ROOT.kRed)

        chist.SetLineWidth(2)
        chist.SetYTitle("Opening Angle / ^{o} ")
        chist.SetXTitle("#splitline{Local Y Position of}{ALPIDE 1 Cluster / [mm]}") 
        legend.AddEntry(chist, labels[index]+ ", #mu= " + str(chist.GetMean())[0:5])
        chist.Draw("COLZ")

        #line1 = ROOT.TLine(-7, p2pOpAng[index], 7, p2pOpAng[index])
        #line1.SetLineColor(6)
        #line1.SetLineWidth(2)
        #line1.SetLineStyle(2)
        #line1.Draw()
        #lines.append(line1)
        ROOT.gStyle.SetPadLeftMargin(0.3)
        ROOT.gPad.SetGrid(3)

        line2 = ROOT.TLine(-7, ppOpAng[index], 7, ppOpAng[index])
        #line2.SetLineColor(ROOT.kRed) #7)
        line2.SetLineWidth(2)
        line2.SetLineStyle(2)
        line2.Draw()
        lines.append(line2)
        pad.Update()
    
canvas.Update()
canvas.Draw()
canvas.SaveAs("BestAlignOpeningAnglesAllEnergies.pdf")

Warning in <TROOT::Append>: Replacing existing TH1: Opening Angle (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: Opening Angle (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: Opening Angle (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: Openning Angle Divided (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: Openning Angle Divided (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: Openning Angle Divided (Potential memory leak).
Info in <TCanvas::Print>: pdf file BestAlignOpeningAnglesAllEnergies.pdf has been created


# Cluster Occupancy

In [298]:
# %jsroot on
detectors = ["ALPIDE_0","ALPIDE_1","ALPIDE_2","ALPIDE_3","ALPIDE_4"]
canvas = ROOT.TCanvas("","",1300,600)
canvas.Divide(2,1)
legend = ROOT.TLegend()

# Calculate histogram
hClusSize = ROOT.TH1F("ALPIDE_occupancy", "Total Cluster Occupancy", 5, -0.5, 4.5)
hClusSize1 = ROOT.TH1F("ALPIDE_occupancy", "7 to 20 px Cluster Occupancy", 5, -0.5, 4.5)

for event in readEvents(path1,-1):
    for cluster in event.clusters:
        hClusSize.Fill(detectors.index(cluster.detector), cluster.size)
        if 20 > cluster.size > 7:
            hClusSize1.Fill(detectors.index(cluster.detector), cluster.size)

# Set bin labels
for i, det in enumerate(detectors):
    hClusSize.GetXaxis().SetBinLabel(i+1, det.replace('_', ' '))
    hClusSize1.GetXaxis().SetBinLabel(i+1, det.replace('_', ' '))

#Draw histogram
canvas.cd(1)

ROOT.gPad.SetGrid(3)
hClusSize.SetLineWidth(2)
hClusSize.SetLineColor(ROOT.kBlack)
hClusSize.SetMinimum(0)
hClusSize.Draw("SAME HIST")
canvas.cd(2)
hClusSize1.SetLineWidth(2)
hClusSize1.SetLineColor(ROOT.kBlack)
hClusSize1.SetMinimum(0)
hClusSize1.Draw("SAME HIST")

ROOT.gPad.SetGrid()
ROOT.gPad.SetGrid(3)
ROOT.gStyle.SetPadLeftMargin(0.15)
ROOT.gStyle.SetPadRightMargin(0.3)
ROOT.gStyle.SetPadBottomMargin(0.25)
ROOT.gStyle.SetPadTopMargin(0.19)
canvas.Draw()

canvas.SaveAs("cluster_occupancy_plots.pdf")

Going to process all events.


Warning in <TROOT::Append>: Replacing existing TH1: ALPIDE_occupancy (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: ALPIDE_occupancy (Potential memory leak).
Info in <TCanvas::Print>: pdf file cluster_occupancy_plots.pdf has been created


In [297]:
canvas.cd(1)

ROOT.gPad.SetGrid(3)
hClusSize.SetLineWidth(2)
hClusSize.SetLineColor(ROOT.kBlack)
hClusSize.SetMinimum(0)
hClusSize.Draw("SAME HIST")
canvas.cd(2)
hClusSize1.SetLineWidth(2)
hClusSize1.SetLineColor(ROOT.kBlack)
hClusSize1.SetMinimum(0)
hClusSize1.Draw("SAME HIST")

ROOT.gPad.SetGrid()
ROOT.gPad.SetGrid(3)
ROOT.gStyle.SetPadLeftMargin(0.15)
ROOT.gStyle.SetPadRightMargin(0.3)
ROOT.gStyle.SetPadBottomMargin(0.25)
ROOT.gStyle.SetPadTopMargin(0.19)
canvas.Draw()

canvas.SaveAs("cluster_occupancy_plots.pdf")

Info in <TCanvas::Print>: pdf file cluster_occupancy_plots.pdf has been created


# Plot Cluster Sizes (Aluminum, Target, And Entire Region)

In [22]:
%jsroot 

detectors = ["ALPIDE_0","ALPIDE_1","ALPIDE_2","ALPIDE_3","ALPIDE_4"]
colors = [1,2,3,4,6,7,8,9]
canvas = ROOT.TCanvas("","",600,600)
legend = ROOT.TLegend(0.8,0.8)
det=0
# Calculate histogram
hClusSize = [ROOT.TH1F("Cluster_Size", "ALPIDE "+str(det)+" at 200 MeV Threshold  100e", 41, -0.5, 40.5) for detr in detectors]
hClusSize1 = [ROOT.TH1F("Cluster_Size", "Vertex Region Cluster Sizes from File "+path2[-10:], 41, -0.5, 40.5) for detr in detectors]

histograms = [hClusSize, hClusSize2]

AluRegion = False
AroundTargetRegion= True

if AluRegion:
    xMin=4.5
    xMax=8

    yMin=-4
    yMax=4

    zMin=-20
    zMax=-17.5
    

elif AroundTargetRegion:
    xMin=-4.5
    xMax=4.5

    yMin=-6
    yMax=6

    zMin=-23
    zMax=-15

xTMin=-2
xTMax=2

yTMin=-5
yTMax=5

zTMin=-20
zTMax=-17


tVolume= (xTMax-xTMin) * (yTMax-yTMin) * (zTMax-zTMin)
volume= (xMax-xMin) * (yMax-yMin) * (zMax-zMin) - tVolume

for hist in histograms:
    for h in hist:
        h.Reset()
for event in eventsTracked[4]:#eventSet:
    for index,detector in enumerate(detectors): #index is the dectector
        for cluster in event.clusters:
            if cluster.detector == detector:
                if( xMin < event.vertex.point[0] < xMax and yMin < event.vertex.point[1] < yMax and zMin < event.vertex.point[2] < zMax):
                    if not( xTMin < event.vertex.point[0] < xTMax and yTMin < event.vertex.point[1] < yTMax and zTMin < event.vertex.point[2] < zTMax):
                        if event.vertex.point[0] > 0:
                            histograms[0][index].Fill(cluster.size, 2/volume)
                        else:
                            histograms[1][index].Fill(cluster.size, 2/volume)   
    
    
                    
#Draw histogram
labels=["x>0","x<0"]
print(len(histograms[1]))
#fits
gauss1 = ROOT.TF1("gauss", "gaus")
gauss2 = ROOT.TF1("gauss", "gaus")

#gauss= [gauss1,gauss2, gauss3, gauss4, gauss5, gauss6, gauss7, gauss8]
canvas.Clear()
for index,hist in enumerate(histograms):
    print("index", index)
    #gauss[index].SetLineColor(colors[index])
    #hist[det].Scale(1/hist[det].Integral())
    hist[det].SetMaximum(0.3)
    hist[det].Draw("Same HIST")
    #hist[det].Fit(gauss[index])

    # retrieve the fit parameters
    #mean = gauss[index].GetParameter(1)
    #sigma = gauss[index].GetParameter(2)
    #sigNormd = sigma/np.sqrt(chist.GetEntries())

    hist[det].SetLineColor(colors[index])
    hist[det].SetMarkerColor(colors[index])
    hist[det].SetMarkerStyle(20)
    hist[det].SetLineWidth(3)
    hist[det].SetXTitle("Cluster size [px]")
    hist[det].SetYTitle("Counts per mm^{3}")
    #legend.AddEntry(hist[det], labels[index]+ " #mu= " + str(mean)[0:5] + " and #sigma/#sqrtN= " + str(sigNormd)[0:5]) 
    legend.AddEntry(hist[det], labels[index]+ ", #mu= " + str(hist[det].GetMean())[0:5]+"px")    
    
    
canvas.Update()
legend.Draw() 
canvas.Draw()
canvas.SaveAs("ClusterSizeLR.pdf")

6
index 0
index 1


Warning in <TROOT::Append>: Replacing existing TH1: Cluster_Size (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: Cluster_Size (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: Cluster_Size (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: Cluster_Size (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: Cluster_Size (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: Cluster_Size (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: Cluster_Size (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: Cluster_Size (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: Cluster_Size (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: Cluster_Size (Potential memory leak).
Warning in <TLegend::Paint>: Legend too large to be automatically placed; a default position is used

In [309]:
%jsroot 

detectors = ["ALPIDE_0","ALPIDE_1","ALPIDE_2","ALPIDE_3","ALPIDE_4"]
colors = [1,2,3,4,6,7,8,9]
canvas = ROOT.TCanvas("","",600,600)
legend = ROOT.TLegend(0.8,0.8)
det=0
# Calculate histogram

title1="Aluminum Region Second Weekend ALPIDE "+str(det) #+" for all Thresholds and Energies"
title2="x < 0 ALPIDE "+str(det)+" at Threshold  100e for Energies  200MeV, 120MeV, and 80MeV "
hClusSize  = [ROOT.TH1F("Cluster_Size", title1, 20, -0.5, 40.5) for detr in detectors]
hClusSize2 = [ROOT.TH1F("Cluster_Size", "Vertex Region Cluster Sizes from File "+path2[-10:], 20, -0.5, 40.5) for detr in detectors]
hClusSize3 = [ROOT.TH1F("Cluster_Size", "Vertex Region Cluster Sizes from File "+path3[-10:], 20, -0.5, 40.5) for detr in detectors]

hClusSize4 = [ROOT.TH1F("Cluster_Size", title1, 20, -0.5, 40.5) for detr in detectors]
hClusSize5 = [ROOT.TH1F("Cluster_Size", "Vertex Region Cluster Sizes from File "+path2[-10:], 20, -0.5, 40.5) for detr in detectors]
hClusSize6 = [ROOT.TH1F("Cluster_Size", "Vertex Region Cluster Sizes from File "+path3[-10:], 20, -0.5, 40.5) for detr in detectors]
hClusSize7 = [ROOT.TH1F("Cluster_Size", "Vertex Region Cluster Sizes from File "+path3[-10:], 20, -0.5, 40.5) for detr in detectors]

histograms = [hClusSize, hClusSize2, hClusSize3, hClusSize4, hClusSize5, hClusSize6, hClusSize6, hClusSize7]

AluRegion = True
TargetRegion=False

if AluRegion:
    xMin=4.5
    xMax=8

    yMin=-4
    yMax=4

    zMin=-20
    zMax=-17.5
    

elif TargetRegion:
    xMin=-4.5
    xMax=4.5

    yMin=-6
    yMax=6

    zMin=-23
    zMax=-15

xTMin=-2
xTMax=2

yTMin=-5
yTMax=5

zTMin=-20
zTMax=-17

volume= (xMax-xMin) * (yMax-yMin) * (zMax-zMin)
tVolume= (xTMax-xTMin) * (yTMax-yTMin) * (zTMax-zTMin)

#eventArr = [eventsUncut, eventsUncut2, eventsUncut3]

for hist, eventSet in enumerate(eventsTracked):
    for event in eventSet:
        for index,detector in enumerate(detectors): #index is the dectector
            for cluster in event.clusters:
                if cluster.detector == detector:
                    if( xMin < event.vertex.point[0] < xMax and yMin < event.vertex.point[1] < yMax and zMin < event.vertex.point[2] < zMax):
                    #if( xTMin < event.vertex.point[0] < xTMax and yTMin < event.vertex.point[1] < yTMax and zTMin < event.vertex.point[2] < zTMax):
                     #       if event.vertex.point[0] < 0:
                        histograms[hist][index].Fill(cluster.size, 1/volume)
                    #if not( xTMin < event.vertex.point[0] < xTMax and yTMin < event.vertex.point[1] < yTMax and zTMin < event.vertex.point[2] < zTMax):
                     #   histograms[hist][index].Fill(cluster.size, 1/tVolume)
    
    
                    
#Draw histogram
labels=["200MeV", "120MeV", "  80MeV","200e, 200MeV", "200e, 120MeV", "200e, 80MeV","300e, 200MeV", "300e, 120MeV"]


#fits
gauss1 = ROOT.TF1("gauss", "gaus")
gauss2 = ROOT.TF1("gauss", "gaus")
gauss3 = ROOT.TF1("gauss", "gaus")
gauss4 = ROOT.TF1("gauss", "gaus")
gauss5 = ROOT.TF1("gauss", "gaus")
gauss6 = ROOT.TF1("gauss", "gaus")
gauss7 = ROOT.TF1("gauss", "gaus")
gauss8 = ROOT.TF1("gauss", "gaus")
#gauss1.SetLineColor(ROOT.kBlack)
#gauss2.SetLineColor(ROOT.kRed)
#gauss3.SetLineColor(ROOT.kGreen)
gauss= [gauss1,gauss2, gauss3, gauss4, gauss5, gauss6, gauss7, gauss8]

for index,hist in enumerate(histograms[0:1]):
    #gauss[index].SetLineColor(colors[index])
    hist[det].Scale(1/hist[det].Integral())
    hist[det].SetMaximum(0.4)
    hist[det].Draw("Same HIST  ")
    #hist[det].Fit(gauss[index])

    # retrieve the fit parameters
    #mean = gauss[index].GetParameter(1)
    #sigma = gauss[index].GetParameter(2)
    #sigNormd = sigma/np.sqrt(chist.GetEntries())

    hist[det].SetLineColor(colors[index])
    hist[det].SetMarkerColor(colors[index])
    hist[det].SetMarkerStyle(20)
    hist[det].SetLineWidth(3)
    hist[det].SetXTitle("Cluster size [px]")
    hist[det].SetYTitle("Cluster Counts / Normalized to Integral")
    #print(hist[det].GetEntries())
    #legend.AddEntry(hist[det], labels[index]+ " #mu= " + str(mean)[0:5] + " and #sigma/#sqrtN= " + str(sigNormd)[0:5]) 
    legend.AddEntry(hist[det], labels[index]+ ", #mu= " + str(hist[det].GetMean())[0:5])    
    #else:
      #  legend.AddEntry(hist[det],"Target Region " + labels[index])
    
    
    
legend.Draw() 
canvas.Draw()
#canvas.SaveAs("ClusterSizeAlu.pdf")

Warning in <TROOT::Append>: Replacing existing TH1: Cluster_Size (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: Cluster_Size (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: Cluster_Size (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: Cluster_Size (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: Cluster_Size (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: Cluster_Size (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: Cluster_Size (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: Cluster_Size (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: Cluster_Size (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: Cluster_Size (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: Cluster_Size (Potential memory leak).
Warning in

In [176]:
%jsroot on
detectors = ["ALPIDE_0","ALPIDE_1","ALPIDE_2","ALPIDE_3","ALPIDE_4","ALPIDE_5"]
#colors = [1,2,3,4,6,7]
colors = [ROOT.kBlue -1, ROOT.kAzure -2, ROOT.kCyan-6, ROOT.kOrange, ROOT.kOrange+7]

canvas = ROOT.TCanvas("","",600,600)
legend = ROOT.TLegend(0.8,0.8)

# Calculate histogram
#hClusSize = [ROOT.TH1F("Cluster_Size", "Vertex Region with Background Subtracted, Cluster Sizes from File "+path2[-10:], 41, -0.5, 40.5) for det in detectors]
hClusSize = [ROOT.TH1F("Cluster_Size", "Target and Background Regions", 26, -0.5, 25.5) for det in detectors]
hClusSize2 = [ROOT.TH1F("Cluster_Size", "Cluster Sizes in Background Region from File "+path[-10:], 26, -0.5, 25.5) for det in detectors]



xAluMin=4.5
xAluMax=8

yAluMin=-4
yAluMax=4

zAluMin=-20
zAluMax=-17.5

xBackMin=-4.5
xBackMax=4.5

yBackMin=-6
yBackMax=6

zBackMin=-23
zBackMax=-15

xTMin=-2
xTMax=2

yTMin=-5
yTMax=5

zTMin=-20
zTMax=-17

tVolume= int((xTMax-xTMin) * (yTMax-yTMin) * (zTMax-zTMin))
backVolume= int((xBackMax-xBackMin) * (yBackMax-yBackMin) * (zBackMax-zBackMin) -tVolume)
Back_clusters=0
Target_clusters=0


for event in eventsTracked[0]: #readEvents(path,1e5): #from path to FlippedEvents
    for cluster in event.clusters:
        for detector in detectors:
            if cluster.detector == detector:
                if( xTMin < event.vertex.point[0] < xTMax and yTMin < event.vertex.point[1] < yTMax and zTMin < event.vertex.point[2] < zTMax):
                    hClusSize[int(cluster.detector[-1])].Fill(cluster.size)#, 1/tVolume)
                    Target_clusters+=1
                elif ( xBackMin < event.vertex.point[0] < xBackMax and yBackMin < event.vertex.point[1] < yBackMax and zBackMin < event.vertex.point[2] < zBackMax):
                    hClusSize2[int(cluster.detector[-1])].Fill(cluster.size)#, 1/backVolume)#5/31)
                    Back_clusters+=1
#Draw histogram
#for index,hist in enumerate(hClusSize):
#index = 4
for index,hist in enumerate(hClusSize[0:5]):
    #hist.Scale(1/hist.Integral())
    #hist.SetMaximum(1000)
    #hist = hClusSize[index]
    hist.Scale(1/tVolume)
    hist.Draw("SAME HIST")#("Same P L ")
    hist.SetLineColor(colors[index])
    hist.SetMarkerColor(colors[index])
    hist.SetMarkerStyle(20)
    hist.SetLineWidth(3)
    hist.SetXTitle("Cluster Size [px]")
    hist.SetYTitle("Counts / mm^{3}") # (normalized to integral)")
    #hist.SetYTitle("#splitline{Counts }{[mm^{-3} Detector^{-1} Cluster Size^{-1}]}") # (normalized to integral)")

    legend.AddEntry(hist,"ALPIDE " + detectors[index][-1])
    #ROOT.gPad.SetLogy()

#if False:    
for index,hist in enumerate(hClusSize2[0:5]):
    #hist.Scale(1/hist.Integral())
        #hist.SetMaximum(0.2)
    hist.Scale(1/backVolume)
    hist.Draw("SAME HIST")#("Same P L")
    hist.SetLineColor(colors[index])
    hist.SetMarkerColor(colors[index])
    hist.SetMarkerStyle(2)
    hist.SetLineWidth(3)
    hist.SetLineStyle(2)
    hist.SetXTitle("Cluster Size [px]")
    hist.SetYTitle("Normalized Counts") # (normalized to integral)")
    #legend.AddEntry(hist,"Background ALPIDE " + detectors[index][-1])
        #ROOT.gPad.SetLogy()
    
#legend.SetHeader("#splitline{Target Solid}{Background Dashed}")#, "C")
legend.SetHeader("Target Solid, Background Dotted")#, "C")
legend.Draw()
canvas.Draw()
ROOT.gPad.SetGrid(2)
print("Target_clusters", Target_clusters)
print("Target_clusters density", Target_clusters/tVolume)
print("Back_clusters", Back_clusters)
print("Back_clusters density", Back_clusters/backVolume)
#canvas.SaveAs("ClusterSizeTargetAndBackground.pdf")

Target_clusters 7975
Target_clusters density 66.45833333333333
Back_clusters 4885
Back_clusters density 6.565860215053763


Warning in <TROOT::Append>: Replacing existing TH1: Cluster_Size (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: Cluster_Size (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: Cluster_Size (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: Cluster_Size (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: Cluster_Size (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: Cluster_Size (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: Cluster_Size (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: Cluster_Size (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: Cluster_Size (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: Cluster_Size (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: Cluster_Size (Potential memory leak).
Warning in

In [219]:
%jsroot on
detectors = ["ALPIDE_0","ALPIDE_1","ALPIDE_2","ALPIDE_3","ALPIDE_4","ALPIDE_5"]
#colors = [1,2,3,4,6,7]

canvas = ROOT.TCanvas("","",600,600)
colors = [ROOT.kBlue -1, ROOT.kAzure -2, ROOT.kCyan-6, ROOT.kOrange, ROOT.kOrange+7]

# Calculate histogram
#hClusSize = [ROOT.TH1F("Cluster_Size", "Vertex Region with Background Subtracted, Cluster Sizes from File "+path2[-10:], 41, -0.5, 40.5) for det in detectors]
energy = [200, 120, 80, 200, 120, 80, 200, 120]
threshold = [100, 100, 100, 200, 200, 200, 200, 300]
hClusSize = [ROOT.TH1F("Cluster_Size", "#splitline{Target with Background Region}{Subtracted, for 100e Threshold}", 26, -0.5, 25.5) for i in range(3)]



xAluMin=4.5
xAluMax=8

yAluMin=-4
yAluMax=4

zAluMin=-20
zAluMax=-17.5

xBackMin=-4.5
xBackMax=4.5

yBackMin=-6
yBackMax=6

zBackMin=-23
zBackMax=-15

xTMin=-2
xTMax=2

yTMin=-5
yTMax=5

zTMin=-20
zTMax=-17

tVolume= int((xTMax-xTMin) * (yTMax-yTMin) * (zTMax-zTMin))
backVolume= int((xBackMax-xBackMin) * (yBackMax-yBackMin) * (zBackMax-zBackMin) -tVolume)
Back_clusters=0
Target_clusters=0

for index, trackedList in enumerate([eventsTracked[i]for i in [0, 1, 2]]):
    for event in trackedList:
        for cluster in event.clusters:
            for detector in detectors:
                if cluster.detector == detector:
                    if( xTMin < event.vertex.point[0] < xTMax and yTMin < event.vertex.point[1] < yTMax and zTMin < event.vertex.point[2] < zTMax):
                        #hClusSize[int(cluster.detector[-1])].Fill(cluster.size, 1/tVolume)
                        hClusSize[index].Fill(cluster.size, 1/tVolume)
                    elif ( xBackMin < event.vertex.point[0] < xBackMax and yBackMin < event.vertex.point[1] < yBackMax and zBackMin < event.vertex.point[2] < zBackMax):
                        hClusSize[index].Fill(cluster.size, -1/backVolume)#5/31)


legend = ROOT.TLegend(0.6,0.8)
#if False:    
for index,hist in enumerate(hClusSize[0:3]):
    hist.Scale(1/hist.Integral())
    hist.SetMaximum(0.3)
    hist.Draw("SAME HIST")#("Same P L")
    hist.SetLineColor(colors[index+2])
    #hist.SetMarkerColor(colors[index])
    #hist.SetMarkerStyle(2)
    hist.SetLineWidth(3)
    #hist.SetLineStyle(2)
    hist.SetXTitle("Cluster Size [px]")
    hist.SetYTitle("Counts / Integral") # (normalized to integral)")
    #legend.AddEntry(hist,str(100* (1+index))+"e") #detectors[index][-1]) # str(energy[index]) + " MeV " + 
    legend.AddEntry(hist,str(energy[index]) + " MeV ")
        #ROOT.gPad.SetLogy()
    
#legend.SetHeader("#splitline{Target Solid}{Background Dashed}")#, "C")
#legend.SetHeader("Target Solid, Background Dotted")#, "C")
legend.SetX1NDC(0.7)
legend.SetY1NDC(0.8)
legend.SetX2NDC(0.9)
legend.SetY2NDC(0.9)
ROOT.gStyle.SetPadTopMargin(0.2)
canvas.Update()
legend.Draw()
canvas.Draw()
ROOT.gPad.SetGrid(2)
print("Target_clusters", Target_clusters)
print("Target_clusters density", Target_clusters/tVolume)
print("Back_clusters", Back_clusters)
print("Back_clusters density", Back_clusters/backVolume)
canvas.SaveAs("ClusterSizeEnergiesFor100eNormalized.pdf")

Target_clusters 0
Target_clusters density 0.0
Back_clusters 0
Back_clusters density 0.0


Warning in <TROOT::Append>: Replacing existing TH1: Cluster_Size (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: Cluster_Size (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: Cluster_Size (Potential memory leak).
Warning in <TLegend::Paint>: Legend too large to be automatically placed; a default position is used
Info in <TCanvas::Print>: pdf file ClusterSizeEnergiesFor100eNormalized.pdf has been created


In [247]:
print(tVolume)
print(backVolume)

120
744


In [21]:
%jsroot on
detectors = ["ALPIDE_0","ALPIDE_1","ALPIDE_2","ALPIDE_3","ALPIDE_4","ALPIDE_5"]
#colors = [1,2,3,4,6,7]
colors = [ROOT.kBlue -1, ROOT.kAzure -2, ROOT.kCyan-6, ROOT.kOrange, ROOT.kOrange+7]

canvas = ROOT.TCanvas("","",600,600)
legend = ROOT.TLegend(0.8,0.8)

# Calculate histogram
#hClusSize = [ROOT.TH1F("Cluster_Size", "Vertex Region with Background Subtracted, Cluster Sizes from File "+path2[-10:], 41, -0.5, 40.5) for det in detectors]
hClusSize = [ROOT.TH1F("Cluster_Size", "Target and Background-Subtracted Target Region", 26, -0.5, 25.5) for det in detectors]
hClusSize2 = [ROOT.TH1F("Cluster_Size", "Cluster Sizes in Background Region from File "+path[-10:], 26, -0.5, 25.5) for det in detectors]



xAluMin=4.5
xAluMax=8

yAluMin=-4
yAluMax=4

zAluMin=-20
zAluMax=-17.5

xBackMin=-4.5
xBackMax=4.5

yBackMin=-6
yBackMax=6

zBackMin=-23
zBackMax=-15

xTMin=-2
xTMax=2

yTMin=-5
yTMax=5

zTMin=-20
zTMax=-17

tVolume= int((xTMax-xTMin) * (yTMax-yTMin) * (zTMax-zTMin))
backVolume= int((xBackMax-xBackMin) * (yBackMax-yBackMin) * (zBackMax-zBackMin) -tVolume)



for event in eventsTracked[0]: #readEvents(path,1e5): #from path to FlippedEvents
    for cluster in event.clusters:
        for detector in detectors:
            #if cluster.detector == detector:
                # if( xTMin < event.vertex.point[0] < xTMax and yTMin < event.vertex.point[1] < yTMax and zTMin < event.vertex.point[2] < zTMax):
                #     hClusSize[int(cluster.detector[-1])].Fill(cluster.size, 1/tVolume)
                #     hClusSize2[int(cluster.detector[-1])].Fill(cluster.size, 1/tVolume)#5/31)
                # elif ( xBackMin < event.vertex.point[0] < xBackMax and yBackMin < event.vertex.point[1] < yBackMax and zBackMin < event.vertex.point[2] < zBackMax):
                #     hClusSize2[int(cluster.detector[-1])].Fill(cluster.size, -1/backVolume)
            if( xTMin < event.vertex.point[0] < xTMax and yTMin < event.vertex.point[1] < yTMax and zTMin < event.vertex.point[2] < zTMax):
                hClusSize[0].Fill(cluster.size, 1/tVolume)
                hClusSize2[0].Fill(cluster.size, 1/tVolume)#5/31)
            elif ( xBackMin < event.vertex.point[0] < xBackMax and yBackMin < event.vertex.point[1] < yBackMax and zBackMin < event.vertex.point[2] < zBackMax):
                hClusSize2[0].Fill(cluster.size, -1/backVolume)
            
#Draw histogram
#for index,hist in enumerate(hClusSize):
#index = 4
for index,hist in enumerate(hClusSize[0:1]):
    #hist.Scale(1/hist.Integral())
    #hist.SetMaximum(1000)
    #hist = hClusSize[index]
    hist.Draw("SAME HIST")#("Same P L ")
    ROOT.gPad.SetGrid(2)
    hist.SetLineColor(colors[index])
    hist.SetMarkerColor(colors[index])
    hist.SetMarkerStyle(20)
    hist.SetLineWidth(3)
    hist.SetXTitle("Cluster Size [px]")
    hist.SetYTitle("Counts on All Detectors / mm^{3}") # (normalized to integral)")
    legend.AddEntry(hist,"Target")# ALPIDE " + detectors[index][-1])
    #ROOT.gPad.SetLogy()

#if False:    
for index,hist in enumerate(hClusSize2[0:1]):
    #hist.Scale(1/hist.Integral())
        #hist.SetMaximum(0.2)
    hist.Draw("SAME HIST")#("Same P L")
    ROOT.gPad.SetGrid(2)
    hist.SetLineColor(ROOT.kGreen)#colors[index+2])
    hist.SetMarkerColor(ROOT.kGreen)#colors[index+2])
    hist.SetMarkerStyle(20)
    hist.SetLineWidth(3)
    #hist.SetLineStyle(2)
    hist.SetXTitle("Cluster Size [px]")
    hist.SetYTitle("Normalized Counts") # (normalized to integral)")
    legend.AddEntry(hist,"#splitline{Background}{Subtracted Target}")# ALPIDE " + detectors[index][-1])
        #ROOT.gPad.SetLogy()
    
legend.Draw() 
canvas.Draw()
canvas.SaveAs("ClusterSizeTargetBackSubtractedAndTarget.pdf")

Warning in <TROOT::Append>: Replacing existing TH1: Cluster_Size (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: Cluster_Size (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: Cluster_Size (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: Cluster_Size (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: Cluster_Size (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: Cluster_Size (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: Cluster_Size (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: Cluster_Size (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: Cluster_Size (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: Cluster_Size (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: Cluster_Size (Potential memory leak).
Warning in

In [117]:
%jsroot on
detectors = ["ALPIDE_0","ALPIDE_1","ALPIDE_2","ALPIDE_3","ALPIDE_4","ALPIDE_5"]
colors = [ROOT.kBlue -1, ROOT.kAzure -2, ROOT.kCyan-6, ROOT.kOrange, ROOT.kOrange+7]

canvas = ROOT.TCanvas("","",1200,900)
canvas.Divide(2,2)

# Calculate histogram
hClusSizeT1 =ROOT.TH1F("Cluster_Size_T1", "Target Region", 31, -0.5, 30.5)
hClusSizeT2 =ROOT.TH1F("Cluster_Size_T2", "Target Region with Background Subtracted", 31, -0.5, 30.5)
hClusSizeA = ROOT.TH1F("Cluster_Size_A", "Aluminum Region", 31, -0.5, 30.5)
hClusSizeB = ROOT.TH1F("Cluster_Size_B", "Cluster Sizes in Background Region", 31, -0.5, 30.5)

xAluMin=4.5
xAluMax=8
yAluMin=-4
yAluMax=4
zAluMin=-20
zAluMax=-17.5

xBackMin=-4.5
xBackMax=4.5
yBackMin=-6
yBackMax=6
zBackMin=-23
zBackMax=-15

xTMin=-2
xTMax=2
yTMin=-5
yTMax=5
zTMin=-20
zTMax=-17

tVolume= int((xTMax-xTMin) * (yTMax-yTMin) * (zTMax-zTMin))
backVolume= int((xBackMax-xBackMin) * (yBackMax-yBackMin) * (zBackMax-zBackMin) -tVolume)

for event in eventsTracked[0]:
    for cluster in event.clusters:
        if cluster.detector == detectors[2]:
            if( xAluMin < event.vertex.point[0] < xAluMax and yAluMin < event.vertex.point[1] < yAluMax and zAluMin < event.vertex.point[2] < zAluMax):
                hClusSizeA.Fill(cluster.size, 1/tVolume)
            if( xTMin < event.vertex.point[0] < xTMax and yTMin < event.vertex.point[1] < yTMax and zTMin < event.vertex.point[2] < zTMax):
                hClusSizeT1.Fill(cluster.size, 1/tVolume)
                hClusSizeT2.Fill(cluster.size, 1/tVolume)
            elif ( xBackMin < event.vertex.point[0] < xBackMax and yBackMin < event.vertex.point[1] < yBackMax and zBackMin < event.vertex.point[2] < zBackMax):
                hClusSizeB.Fill(cluster.size, 1/backVolume)
                hClusSizeT2.Fill(cluster.size, -1/backVolume)

stack1 = ROOT.THStack("stack1", "Target Region")
stack2 = ROOT.THStack("stack2", "Target Region with Background Subtracted")
stack3 = ROOT.THStack("stack3", "Aluminum Region")
stack4 = ROOT.THStack("stack4", "Cluster Sizes in Background Region")

hist = hClusSizeT1
ROOT.gPad.SetGrid(2)
hist.SetLineColor(colors[0])
hist.SetMarkerColor(colors[0])
hist.SetMarkerStyle(20)
hist.SetLineWidth(3)
hist.Scale(1/hist.Integral())
hist.SetXTitle("Cluster Size [px]")
hist.SetYTitle("Counts per mm^{3}")
stack1.Add(hist)

hist = hClusSizeT2
hist.SetLineColor(colors[0])
hist.SetMarkerColor(colors[0])
hist.SetMarkerStyle(20)
hist.SetLineWidth(3)
hist.Scale(1/hist.Integral())
hist.SetXTitle("Cluster Size [px]")
hist.SetYTitle("Counts per mm^{3}")
stack2.Add(hist)

hist = hClusSizeA
hist.SetLineColor(colors[0])
hist.SetMarkerColor(colors[0])
hist.SetMarkerStyle(20)
hist.SetLineWidth(3)
hist.Scale(1/hist.Integral())
hist.SetXTitle("Cluster Size [px]")
hist.SetYTitle("Counts per mm^{3}")
stack3.Add(hist)

hist = hClusSizeB
hist.SetLineColor(colors[0])
hist.SetMarkerColor(colors[0])
hist.SetMarkerStyle(20)
hist.SetLineWidth(3)
hist.Scale(1/hist.Integral())
stack4.Add(hist)
index=0





canvas.cd(1)
ROOT.gPad.SetGrid(2)
ROOT.gStyle.SetPadRightMargin(0.02)
stack1.Draw("nostack HIST")
stack1.GetXaxis().SetTitleOffset(1.2) # Adjust x-axis title position
stack1.GetYaxis().SetTitleOffset(1.2) # Adjust y-axis title position
stack1.GetXaxis().SetTitle("Cluster Size [px]")
stack1.GetYaxis().SetTitle("Counts Normalized to Integral")

canvas.cd(2)
ROOT.gPad.SetGrid(2)
ROOT.gStyle.SetPadRightMargin(0.02)
stack4.Draw("nostack HIST")
stack4.GetXaxis().SetTitleOffset(1.2) # Adjust x-axis title position
stack4.GetYaxis().SetTitleOffset(1.2) # Adjust y-axis title position
stack4.GetXaxis().SetTitle("Cluster Size [px]")
stack4.GetYaxis().SetTitle("Counts Normalized to Integral")

canvas.cd(3)
ROOT.gPad.SetGrid(2)
stack2.Draw("nostack HIST")
stack2.GetXaxis().SetTitleOffset(1.2) # Adjust x-axis title position
stack2.GetYaxis().SetTitleOffset(1.2) # Adjust y-axis title position
stack2.GetXaxis().SetTitle("Cluster Size [px]")
stack2.GetYaxis().SetTitle("Counts Normalized to Integral")

canvas.cd(4)
ROOT.gPad.SetGrid(2)
stack3.Draw("nostack HIST")
stack3.GetXaxis().SetTitleOffset(1.2) # Adjust x-axis title position
stack3.GetYaxis().SetTitleOffset(1.2) # Adjust y-axis title position
stack3.GetXaxis().SetTitle("Cluster Size [px]")
stack3.GetYaxis().SetTitle("Counts Normalized to Integral")

canvas.cd(4)
ROOT.gPad.SetGrid(2)
stack4.Add(hist)
index=0
canvas.cd(1)
ROOT.gPad.SetGrid(2)
stack1.Draw("nostack HIST")
legend1 = ROOT.TLegend(0.8,0.8)
legend1.AddEntry(hClusSizeT1,"ALPIDE " + detectors[index][-1])
#legend1.Draw()

canvas.cd(2)
ROOT.gPad.SetGrid(2)
ROOT.gStyle.SetPadRightMargin(0.02)
stack4.Draw("nostack HIST")
legend4 = ROOT.TLegend(0.8,0.8)
legend4.AddEntry(hist,"ALPIDE " + detectors[index][-1])
#legend4.Draw()

canvas.cd(3)
ROOT.gPad.SetGrid(2)
ROOT.gStyle.SetPadRightMargin(0.02)
stack2.Draw("nostack HIST")
legend2 = ROOT.TLegend(0.8,0.8)
legend2.AddEntry(hist,"ALPIDE " + detectors[index][-1])
#legend2.Draw()

canvas.cd(4)
ROOT.gPad.SetGrid(2)
ROOT.gStyle.SetPadRightMargin(0.02)
stack3.Draw("nostack HIST")
legend3 = ROOT.TLegend(0.8,0.8)
legend3.AddEntry(hist,"ALPIDE " + detectors[index][-1])
#legend3.Draw()

canvas.Draw()
canvas.SaveAs("AllClusterSize.pdf")



Warning in <TROOT::Append>: Replacing existing TH1: Cluster_Size_T1 (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: Cluster_Size_T2 (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: Cluster_Size_A (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: Cluster_Size_B (Potential memory leak).
Info in <TCanvas::Print>: pdf file AllClusterSize.pdf has been created


In [151]:
%jsroot on
# plot RMS of distances between clusters and the track
hRMS = ROOT.TH1F("RMS cluster","RMS Cluster",22*5,-0.1,2.1)
hRMS.GetXaxis().SetTitle("RMS(cluster distance in track) [mm]")
hRMS.GetYaxis().SetTitle("Count")

for event in eventsTracked[0]:
    hRMS.Fill(event.tracks[1].rms)


canvas = ROOT.TCanvas()
canvas.SetGridx(2)
hRMS.Draw("")
canvas.Draw()

# Plot Correlation

In [152]:
%jsroot on

nBins = {"x-Axis" : 300, "y-Axis" : 150}
nRange = {"x-Axis" : 15, "y-Axis" : 7.5}
axis = {"x-Axis":0, "y-Axis":1}

detector1 = "ALPIDE_0"
detector2 = "ALPIDE_4"
axis1="x-Axis"
axis2="y-Axis"
lowerClustSizThresh = 7  #thresholds for how big to make the file for analysis
upperClustSizThresh = 15
clustNumThresh = 1
    
hCorr = ROOT.TH2F("Corr_"+detector1+"_"+axis1+"_"+detector2+"_"+axis2,"Correlation", nBins.get(axis1),-nRange.get(axis1),nRange.get(axis1),nBins.get(axis2),-nRange.get(axis2),nRange.get(axis2))
hCorr.SetXTitle(detector1 + " - " + axis1)
hCorr.SetYTitle(detector2 + " - " + axis2)

for event in eventsTracked[0]: #readEvents(path2, nEvents=1e5):
    clustersDetector1 = event.selectDetector(detector1)
    clustersDetector2 = event.selectDetector(detector2)
    #selectedClusters1 = [cluster for cluster in clustersDetector1 if upperClustSizThresh > cluster.size > lowerClustSizThresh]
    #selectedClusters2 = [cluster for cluster in clustersDetector2 if upperClustSizThresh > cluster.size > lowerClustSizThresh]
    #if len(selectedClusters1) <= clustNumThresh and len(selectedClusters2) <= clustNumThresh:
    for cluster1 in clustersDetector1:
        for cluster2 in clustersDetector2:
            hCorr.Fill(cluster1.localPos[axis.get(axis1)],cluster2.localPos[axis.get(axis2)])
                

canvas = ROOT.TCanvas()
hCorr.Draw("COLZ")
canvas.Draw()

In [153]:
%jsroot on

nBins = {"x-Axis" : 500, "y-Axis" : 500}
nRange = {"x-Axis" : 15, "y-Axis" : 7.5}
axis = {"x-Axis":0, "y-Axis":1}
detectors = ["ALPIDE_0","ALPIDE_1","ALPIDE_2","ALPIDE_3","ALPIDE_4","ALPIDE_5"]

detector1 = "ALPIDE_3"
detector2 = "ALPIDE_4"
#ALPIDE 0 vs ALPIDE 4 
hClusSize = [ROOT.TH2F("Cluster_Size", "Cluster Sizes Vertex Region Cluster Sizes from File "+path2[-10:], 41, -0.5, 40.5,41, -0.5, 40.5) for det in detectors]

for event in eventsTracked[0]: #readEvents(path, nEvents=-1):
    clustersDetector1 = event.selectDetector(detector1)
    clustersDetector2 = event.selectDetector(detector2)
    for cluster1 in clustersDetector1:
        for cluster2 in clustersDetector2:
            hClusSize[0].Fill(cluster1.size,cluster2.size)
                

canvas = ROOT.TCanvas()
#hClusSize[0].Scale(1/hClusSize[0].Integral())
#hclusSize[0].SetMaximum(0.2)
hClusSize[0].Draw("COLZ")
#hClusSize[0].SetLineColor(colors[2])
#hClusSize[0].SetMarkerColor(colors[2])
#hClusSize[0].SetMarkerStyle(20)
#hClusSize[0].SetLineWidth(3)
#hClusSize[0].SetLineStyle(20)
hClusSize[0].SetXTitle("Cluster size ALPIDE 0 / [pixels]")
hClusSize[0].SetYTitle("Cluster size ALPIDE 4 / [pixels]") # (normalized to integral)")
legend.AddEntry(hClusSize[0],"Background " + detectors[index])
canvas.Draw()

Warning in <TROOT::Append>: Replacing existing TH1: Cluster_Size (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: Cluster_Size (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: Cluster_Size (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: Cluster_Size (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: Cluster_Size (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: Cluster_Size (Potential memory leak).


## Plot Beam Energy vs Cluster Count vs Mean Free Path

In [155]:
%jsroot on
detectors = ["ALPIDE_0","ALPIDE_1","ALPIDE_2","ALPIDE_3","ALPIDE_4","ALPIDE_5"]
colors = [2,3,4,6,7,8,9]
canvas = ROOT.TCanvas("","",600,600)
legend = ROOT.TLegend()

# Calculate histogram
det=2

hCluster1 = ROOT.TH3F("Cluster_Size", "Beam Energy vs. Cluster Size vs. Track Length through ALPIDE "+str(det), 28, 70, 210, 41, -0.5, 40.5, 200, 20, 100)
#hCluster2 = ROOT.TH3F("Cluster_Size", "Beam Energy vs. Cluster Size vs. Track Length through ALPIDE "+str(det), 28, 70, 210, 41, -0.5, 40.5, 200, 20, 100)
#hCluster3 = ROOT.TH3F("Cluster_Size", "Beam Energy vs. Cluster Size vs. Track Length through ALPIDE "+str(det), 28, 70, 210, 41, -0.5, 40.5, 200, 20, 100)

#histograms = [hCluster1, hCluster2, hCluster3] #three beam energies

xMin=4.5
xMax=8

yMin=-4
yMax=4

zMin=-20
zMax=-17.5

xTMin=-2
xTMax=2

yTMin=-5
yTMax=5

zTMin=-20
zTMax=-17

volume= (xMax-xMin) * (yMax-yMin) * (zMax-zMin)
tVolume= (xTMax-xTMin) * (yTMax-yTMin) * (zTMax-zTMin)

energies=[200, 120, 80]
for hist, eventSet in enumerate(eventsTracked):
    if hist < 3:
        for event in eventSet:
            pathLenSum=0
            numClusters=0    
            for cluster in event.clusters:
                trackSide=0
                numClusters+=1
                if (cluster.detector in alpidesLeft): #remember we are defining a normal vector for the cylinder. here we get the detector and have a radius
                    trackSide=1

                normalVector=[cluster.globalPos[0],cluster.globalPos[1],0 ] #cluster coords perpendicular to z
                normalVector/=np.linalg.norm(normalVector)
                trackVector=event.tracks[trackSide].vector/np.linalg.norm(event.tracks[trackSide].vector) #np.linalg.norm should return the magnitude
                pathLength=0.025/np.dot(normalVector,trackVector)*1000 #get hypotenus (track length through det)convert mm to micron
                pathLenSum+=np.abs(pathLength)
            #for cluster in event.clusters:    
                hCluster1.Fill(energies[hist], cluster.size, np.abs(pathLength)) #,cluster.size, 1/tVolume)
    
    
                    
#Draw histogram
labels=["200MeV", "120MeV", "80MeV","200e, 200MeV", "200e, 120MeV", "200e, 80MeV","300e, 200MeV", "300e, 120MeV"]


#fits
gauss1 = ROOT.TF1("gauss", "gaus")
gauss2 = ROOT.TF1("gauss", "gaus")
gauss3 = ROOT.TF1("gauss", "gaus")


gauss= [gauss1,gauss2, gauss3]

#for index,hist in enumerate(histograms):
    #gauss[index].SetLineColor(colors[index])
    #hist[det].Fit(gauss[index])

    # retrieve the fit parameters
    #mean = gauss[index].GetParameter(1)
    #sigma = gauss[index].GetParameter(2)

    #hist[det].Scale(1/hist[det].Integral())
    #hist[det].SetMaximum(0.3)
    
    #hist[det].SetLineColor(colors[index])
ROOT.gPad.SetGrid(2)
#hCluster1[det].SetMarkerColor(colors[index])
#hCluster1[det].SetMarkerStyle(20)
#legend.AddEntry(hCluster1[det], labels[index])#+ " #mu " + str(mean)[0:3] + " and #sigma " + str(sigma)[0:3]) 


hCluster1.Draw("BOX3")
hCluster1.SetXTitle("Beam Energy / MeV")
hCluster1.SetYTitle("Cluster Size / [pixels]")
hCluster1.SetZTitle("Track Length through ALPIDE "+str(det)+" / Micron")



canvas.Draw()

Warning in <TROOT::Append>: Replacing existing TH1: Cluster_Size (Potential memory leak).


In [215]:
%jsroot on
detectors = ["ALPIDE_0","ALPIDE_1","ALPIDE_2","ALPIDE_3","ALPIDE_4","ALPIDE_5"]
colors = [2,3,4,6,7,8,9]
canvas = ROOT.TCanvas("","",600*3,600)
legend = ROOT.TLegend()

# Calculate histogram
det=0
xStart=5
xFinish=15
yStart=30
yFinish=45
xbin=10
ybin=15*5
hCluster1 = ROOT.TH2F("Cluster_Size", "Beam Energy 200 MeV through ALPIDE "+str(det),xbin, xStart, xFinish, ybin, yStart, yFinish) # -0.5, 40.5, 200, 25, 55) 
hCluster2 = ROOT.TH2F("Cluster_Size", "Beam Energy 120 MeV through ALPIDE "+str(det),xbin, xStart, xFinish, ybin, yStart, yFinish) # -0.5, 40.5, 200, 25, 55) 
hCluster3 = ROOT.TH2F("Cluster_Size", "Beam Energy 200 MeV through ALPIDE "+str(det),xbin,xStart, xFinish, ybin, yStart, yFinish) # -0.5, 40.5, 200, 25, 55) 

histograms = [hCluster1, hCluster2, hCluster3] #three beam energies

xMin=4.5
xMax=8

yMin=-4
yMax=4

zMin=-20
zMax=-17.5

xTMin=-2
xTMax=2

yTMin=-5
yTMax=5

zTMin=-20
zTMax=-17

volume= (xMax-xMin) * (yMax-yMin) * (zMax-zMin)
tVolume= (xTMax-xTMin) * (yTMax-yTMin) * (zTMax-zTMin)

energies=[200, 120, 80]
for hist, eventSet in enumerate(eventsTracked):
    if hist < 3:
        for event in eventSet:
            #pathLenSum=0
            numClusters=0    
            for cluster in event.clusters:
                #if( xMin < event.vertex.point[0] < xMax and yMin < event.vertex.point[1] < yMax and zMin < event.vertex.point[2] < zMax):
                if( xTMin < event.vertex.point[0] < xTMax and yTMin < event.vertex.point[1] < yTMax and zTMin < event.vertex.point[2] < zTMax):
                    trackSide=0
                    numClusters+=1
                    if (cluster.detector in alpidesLeft): #remember we are defining a normal vector for the cylinder. here we get the detector and have a radius
                        #trackSide=1

                        normalVector=[cluster.globalPos[0],cluster.globalPos[1],0 ] #cluster coords perpendicular to z
                        normalVector/=np.linalg.norm(normalVector)
                        trackVector=event.tracks[trackSide].vector/np.linalg.norm(event.tracks[trackSide].vector) #np.linalg.norm should return the magnitude
                        pathLength=0.025/np.dot(normalVector,trackVector)*1000 #get hypotenus (track length through det)convert mm to micron
                        #pathLenSum+=np.abs(pathLength) 
                        histograms[hist].Fill(cluster.size, np.abs(pathLength),1/volume) #,cluster.size, 1/tVolume)
    

                    
#Draw histogram

canvas.Divide(3)
for index,hist in enumerate(histograms):
    canvas.cd(index+1)
    hist.Draw("COLZ")
    hist.SetXTitle("Cluster Size / [pixels]")
    hist.SetYTitle("Track Length through ALPIDE / #mum")
canvas.Draw()
#canvas.SaveAs("ClusterSizevsTrackLength.pdf")


NameError: name 'alpidesLeft' is not defined

Warning in <TROOT::Append>: Replacing existing TH1: Cluster_Size (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: Cluster_Size (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: Cluster_Size (Potential memory leak).


In [158]:
# Create profile
hProfile1 = hCluster1.ProfileX()
canvas = ROOT.TCanvas()

hProfile1.Draw()
canvas.Draw()

## Cluster Size Fit Mean vs. Energy

In [196]:
%jsroot on
detectors = ["ALPIDE_0","ALPIDE_1","ALPIDE_2","ALPIDE_3","ALPIDE_4"]
colors = [2,3,4,6,7,8,9]
canvas = ROOT.TCanvas("","",600,600)

# Create legend
legend = ROOT.TLegend(0.6,0.8,0.6,0.8)

# Calculate histogram
hCluster1 = ROOT.TH2F("Cluster_Size", "Cluster Size Fit Mean vs. Energy", 300, 70, 210, 21, -0.5, 20.5)
hCluster2 = ROOT.TH2F("Cluster_Size", "", 300, 70, 210, 21, -0.5, 20.5)
hCluster3 = ROOT.TH2F("Cluster_Size", "", 300, 70, 210.5, 21, -0.5, 20.5)

graph1 = ROOT.TGraphErrors()
graph2 = ROOT.TGraphErrors()
graph3 = ROOT.TGraphErrors()

histograms = [hCluster1, hCluster2, hCluster3] #three beam energies
graphs=[graph1, graph2, graph3]

energies=[200, 120, 80]

sigmas1=[1.8,2.1,2.1]
sigmas2=[1.7,1.7,1.8]
sigmas3=[0,1.5,0]
sigmas= [sigmas1,sigmas2,sigmas3]

fitMeanClusterSize100e=[9.9, 13, 15]
fitMeanClusterSize200e=[7.9, 10, 12]
fitMeanClusterSize300e=[-1, 8.2, -1]
fitMeanClusterSizes = [fitMeanClusterSize100e, fitMeanClusterSize200e, fitMeanClusterSize300e]

for thresh, meanClustSize in enumerate(fitMeanClusterSizes):
    for energy, meanSize in enumerate(meanClustSize):
        #histograms[thresh].Fill(energies[energy], meanSize)
        graphs[thresh].SetPoint(energy, energies[energy], meanSize)
        graphs[thresh].SetPointError(energy, 0, sigmas[thresh][energy])
    
                    
#Draw histogram
labels=["100e", "200e","300e"]
det=0


# Set bin labels for x-axis


for index,hist in enumerate(histograms):
    #hist[det].Scale(1/hist[det].Integral())
    graphs[index].SetMinimum(5)
    
    hist.SetMarkerColor(colors[index])
    hist.SetMarkerStyle(20)
    graphs[index].SetMarkerColor(colors[index])
    graphs[index].SetMarkerStyle(20)
    hist.Draw("same P L")
    graphs[index].Draw("same P")
    ROOT.gPad.SetGrid(2)
    ROOT.gStyle.SetLabelSize(0.04, "X")  # Increase X-axis label font size
    hist.SetXTitle("Beam Energy / MeV")
    hist.SetYTitle("Mean Cluster Size / pixels")
    legend.AddEntry(graphs[index], labels[index])
canvas.Update()
legend.Draw("COL") 
canvas.Draw("COLZ")
canvas.SaveAs("Cluster_Size_Fit_Mean_Vs_Energy.pdf")

Warning in <TROOT::Append>: Replacing existing TH1: Cluster_Size (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: Cluster_Size (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: Cluster_Size (Potential memory leak).
Warning in <TLegend::Paint>: Legend too large to be automatically placed; a default position is used
Info in <TCanvas::Print>: pdf file Cluster_Size_Fit_Mean_Vs_Energy.pdf has been created


## Kinematic Plots

In [287]:

%jsroot on
hVertexRoman_Plane = ROOT.TH2F("ALPIDE Displacement","ALPIDE Displacement",200,-2,2,200,-2,2)
hVertexRoman_Plane.GetXaxis().SetTitle("X Displacement [mm]")
hVertexRoman_Plane.GetYaxis().SetTitle("Y Displacement [mm]")

h1DA0 = ROOT.TH1F("ALPIDE 0 Displacement [mm]","ALPIDE Displacement [mm]",300,  -2.75,2.75)
h1DA1 = ROOT.TH1F("ALPIDE 1 Displacement [mm]","ALPIDE 1 Displacement [mm]",300,-2.75,2.75)
h1DA2 = ROOT.TH1F("ALPIDE 2 Displacement [mm]","ALPIDE 2 Displacement [mm]",300,-2.75,2.75)
h1DA3 = ROOT.TH1F("ALPIDE 3 Displacement [mm]","ALPIDE 3 Displacement [mm]",300,-2.75,2.75)
h1DA4 = ROOT.TH1F("ALPIDE 4 Displacement [mm]","ALPIDE 4 Displacement [mm]",300,-2.75,2.75)
plots=[[],[],[],[],[],[]]
for j in range(6):
    plots[j] = copy.deepcopy([h1DA0,h1DA1,h1DA2,h1DA3,h1DA4])
#colors=[ROOT.kRed, ROOT.kBlue, ROOT.kGreen, ROOT.kBlack, ROOT.kMagenta]
#colors=[1,2,3,4,6,7,8,9]
canvas = ROOT.TCanvas("canvas", "Multiple Plots", 1800, 900)
legend = ROOT.TLegend(0.6, 0.7, 0.9, 0.9)


    
    
ALPIDES=["ALPIDE_0","ALPIDE_1","ALPIDE_2","ALPIDE_3","ALPIDE_4"]

def distance_to_plane(point, plane_normal, plane_d):
    distance = (np.dot(point, plane_normal) + plane_d)/ np.linalg.norm(plane_normal)
    
    return distance
        
def romanDisplacement(displacement,twistDisplacement, DCA_calc,A1,A2,A3,plotNum):
    dist_arr0= []
    dist_arr1= []
    dist_arr2= []
    dist_arr3= []
    dist_arr4= []
    title_str="Plane formed by ALPIDEs: "+A1[7]+", " + A2[7]+", and " + A3[7]
    plots[plotNum][0].SetTitle(title_str)
    dist_arr = [dist_arr0,dist_arr1,dist_arr2,dist_arr3,dist_arr4]
    
    dcaVals = []
    trackRMSVals = []
    tempEvents=copy.deepcopy(loadEvents("/home/cehrich/Software/uits3_krakow22/cleanEvents_brute.pkl")) #problem is that the events are changed in the algorithm so a copy must be used
    
    for event in tempEvents:
        if(DCA_calc):
            TrackLeft = Track()
            TrackLeft.fromClusters([event.selectDetector(alpide)[0].alignLocal(displacement.get(alpide)[0:3] ) for alpide in ["ALPIDE_3","ALPIDE_4"]])
            TrackLeft.fromClusters([event.selectDetector(alpide)[0].twistTopDown(twistDisplacement[index+3]) for index, alpide in enumerate(["ALPIDE_3","ALPIDE_4"])])  #angles

   
            TrackRight = Track()
            TrackRight.fromClusters([event.selectDetector(alpide)[0].alignLocal(displacement.get(alpide)[0:3]) for alpide in ["ALPIDE_0","ALPIDE_1","ALPIDE_2"]]) 
            TrackRight.fromClusters([event.selectDetector(alpide)[0].twistTopDown(twistDisplacement[index]) for index, alpide in enumerate(["ALPIDE_0","ALPIDE_1","ALPIDE_2"])]) #angles

            vertex = Vertex()
            vertex.fromTracks([TrackLeft,TrackRight])
            dcaVals.append(vertex.dca)
        for alpide in ALPIDES:
            event.selectDetector(alpide)[0].alignLocal(displacement.get(alpide)[0:5])
        v1 = np.array(event.selectDetector(A1)[0].globalPos)- np.array(event.selectDetector(A2)[0].globalPos)
        v2 = np.array(event.selectDetector(A1)[0].globalPos)- np.array(event.selectDetector(A3)[0].globalPos)
        
        norm = [v1[1] * v2[2] - v1[2] * v2[1],
                v1[2] * v2[0] - v1[0] * v2[2],
                v1[0] * v2[1] - v1[1] * v2[0]]
        
        point = np.array(event.selectDetector(A1)[0].globalPos)
        
        D = -(norm[0] * point[0] + norm[1] * point[1] + norm[2] * point[2])
        dist=[distance_to_plane(np.array(event.selectDetector(alpide)[0].globalPos), norm, D) for alpide in ["ALPIDE_0","ALPIDE_1","ALPIDE_2","ALPIDE_3","ALPIDE_4"]]

        
        for i, plot in enumerate(plots[plotNum]):
            plot.Fill(dist[i])
            dist_arr[i].append(dist[i])
            #if
            #legend.AddEntry(plot, "ALPIDE " + str(i), "l")

        #hVertexRoman_Plane.Fill(dist1,dist2)
        
        
        

    if(DCA_calc):   
        for track in event.tracks:
            if track.nClusters == 3:
                trackRMS = track.rms
                trackRMSVals.append(trackRMS)
        DCAnoTail = [dca <= np.median(dcaVals)+ np.std(dcaVals) for dca in dcaVals]
        medianDCA = np.median(dcaVals)
        medianTrackRMS = np.median(trackRMSVals)
        meanDCA = np.mean(dcaVals)
        quality = meanDCA + medianTrackRMS/2
    
        print("Evaluated displacement" ,displacement ,". Got median DCA ",str(medianDCA)[:10],". Got median track RMS ",str(medianTrackRMS)[:10], " and mean DCA: ", str(meanDCA)[:5], " and DCA sigma x,y,z: ", str(np.std(dcaVals[0]))[:5],", ", str(np.std(dcaVals[1]))[:5],", ", str(np.std(dcaVals[2]))[:5], file=f)
    
        return [meanDCA, medianDCA, medianTrackRMS, quality] #try subtracting the median openning angle from the actual openning angle additionally to assess quality
    
    return [title_str, *(np.median(Adist) for Adist in dist_arr)]

null_displacement={
     'ALPIDE_0': [0, 0, 0], 
     'ALPIDE_1': [0, 0, 0], 
     'ALPIDE_2': [0, 0, 0], 
     'ALPIDE_3': [0, 0, 0], 
     'ALPIDE_4': [0, 0, 0]
}
custom_displacement={
     'ALPIDE_0': [0, 0, 0], 
     'ALPIDE_1': [1.1, 0, 0], 
     'ALPIDE_2': [0.2, 0, 0], 
     'ALPIDE_3': [0, 0, 0], 
     'ALPIDE_4': [0, 0, 0]
}

alignment01092023={
     'ALPIDE_0': [-0.28125, -0.225, 0.15], 
     'ALPIDE_1': [-0.225, -0.24375, 0.15], 
     'ALPIDE_2': [0.3, -0.24375, 0.075], 
     'ALPIDE_3': [-0.3, -0.2625, 0.075], 
     'ALPIDE_4': [-0.3, -0.24375, 0.15]
}

best_displacement_no_multiple_scat ={ #10D no anchor dont use
    'ALPIDE_0': [0.5, -0.0546875, 0], 
    'ALPIDE_1': [0.65, -0.0546875, 0], 
    'ALPIDE_2': [0.75, -0.06875, 0], 
    'ALPIDE_3': [0.2, -0.125, 0], 
    'ALPIDE_4': [0.5546875, -0.05, 0]
}



berkin_ml_displacement={
    'ALPIDE_0': [-0.20006977021694183, -0.8966014385223389,0], 
    'ALPIDE_1': [0.7433865666389465, -0.8977992534637451,0], 
    'ALPIDE_2': [-0.8155627250671387, -0.9625105261802673,0], 
    'ALPIDE_3': [0.10266361385583878, 0.31308597326278687,0], 
    'ALPIDE_4': [0.4219255745410919, 1.322750449180603,0]
}
temp_x_align={
    'ALPIDE_0': [0, -1.75, 0], 
    'ALPIDE_1': [1.1, 1.296875, 0], 
    'ALPIDE_2': [0.2, 1.0, 0], 
    'ALPIDE_3': [0, 1.75, 0], 
    'ALPIDE_4': [0, -1.25, 0]
}
dca_A1A2prealigned_with_completeDCA_displacement_fixed_A0const={
    'ALPIDE_0': [0, 0, 0], 
    'ALPIDE_1': [1.1046875, 0.014648437499999998, 0], 
    'ALPIDE_2': [0.19062500000000002, 0.0046875, 0],
    'ALPIDE_3': [0.009375, -0.0046875, 0], 
    'ALPIDE_4': [0.0046875, 0.0087890625, 0]
}
A1_fixed_A1_A2_prealigned={ 
    'ALPIDE_0': [0.0, -0.0005859375, 0], 
    'ALPIDE_1': [1.104541, 0.0149414062, 0], 
    'ALPIDE_2': [0.2, 0.0, 0], 
    'ALPIDE_3': [0.0, -0.0035156249999999997, 0], 
    'ALPIDE_4': [0.0, -0.209765625, 0]
}
singe_det_alignement={  
        "ALPIDE_0" : [0,-1,0],
        "ALPIDE_1" : [1.03125, -0.96875, 0],
        "ALPIDE_2" : [0.0, 0.15625, 0],
        "ALPIDE_3" : [0.1875, 0.1875,0],
        "ALPIDE_4" : [0.0, -0.03125, 0],    
}
my12DAlignment={
    'ALPIDE_0': [0, 0, 0, 0], 
    'ALPIDE_1': [1.10908203125, 0.0369140625, 0, 0.53125], 
    'ALPIDE_2': [0.19882812500000002, 0.00029296875, 0, -0.0009765625], 
    'ALPIDE_3': [0.001171875, -0.00029296875, 0, -0.03125],
    'ALPIDE_4': [0.009375, 0.00029296875, 0, 0.0302734375]
}
Twist12DAlignmentDAngles = [v[-1][-1] for v in my12DAlignment.items()]
localx_twist_alignmentA0fixed = [0, 0.0, 0.0, -0.14012432098388672, 0.0]
localx_twist_alignmentA1fixed = [0.0546875, 0, 0.031249046325683594, 0.0014047622680664062, 0.0625]


displacement=null_displacement
twist= [0,0,0,0,0]

A024=romanDisplacement(displacement,twist, False, "ALPIDE_0", "ALPIDE_2", "ALPIDE_4",0)
A012=romanDisplacement(displacement,twist, False, "ALPIDE_1", "ALPIDE_2", "ALPIDE_4",1)
A124=romanDisplacement(displacement,twist, False, "ALPIDE_0", "ALPIDE_2", "ALPIDE_3",2)
A234=romanDisplacement(displacement,twist, False, "ALPIDE_1", "ALPIDE_2", "ALPIDE_3",3)
A013=romanDisplacement(displacement,twist, False, "ALPIDE_0", "ALPIDE_3", "ALPIDE_4",4)
A134=romanDisplacement(displacement,twist, False, "ALPIDE_2", "ALPIDE_3", "ALPIDE_4",5)
A=[0,0,0,0,0]
A[0]=A024[1]+A012[1]+A124[1]+A234[1]+A013[1]+A134[1]
A[1]=A024[2]+A012[2]+A124[2]+A234[2]+A013[2]+A134[2]
A[2]=A024[3]+A012[3]+A124[3]+A234[3]+A013[3]+A134[3]
A[3]=A024[4]+A012[4]+A124[4]+A234[4]+A013[4]+A134[4]
A[4]=A024[5]+A012[5]+A124[5]+A234[5]+A013[5]+A134[5]
digits=""
mean=0
for a in A:
    mean+=np.abs(a)
    digits = digits + " & "+str(a)[:7] 
mean/=5
print(digits+" & "+str(mean)[:7])
print("mean "+str(mean)[:7])
#Plane formed by str.get()
canvas.Divide(3,2)
legends = [copy.deepcopy(legend) for _ in range(6)]
ROOT.gStyle.SetPadLeftMargin(0.2)
ROOT.gStyle.SetPadRightMargin(0.2)
for j in range(6):
    canvas.cd(j+1)
    ROOT.gPad.SetGrid(2)
    for i,plot in enumerate(plots[j]): #we want a plot for all 6 of these
        plot.SetXTitle("Displacement [mm]")
        plot.SetYTitle("Counts")
        plot.Draw("SAME")
        plot.SetLineColor(colors[i])#(colors[i])
        plot.SetLineWidth(2)
        plot.SetMaximum(300)
        legend_entry=legends[j].AddEntry(plot, "ALPIDE " + str(i), "L")
        legend_entry.SetLineColor(colors[i])
        plot.SetStats(0)
    legends[j].Draw()
canvas.Update()
canvas.Draw()
canvas.SaveAs("kinematic_null_Displacement_DCA.pdf")

 & -4.3993 & 4.02854 & 0.19246 & 0.86993 & -1.4625 & 2.19057
mean 2.19057


Warning in <TROOT::Append>: Replacing existing TH1: ALPIDE Displacement (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: ALPIDE 0 Displacement [mm] (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: ALPIDE 1 Displacement [mm] (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: ALPIDE 2 Displacement [mm] (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: ALPIDE 3 Displacement [mm] (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: ALPIDE 4 Displacement [mm] (Potential memory leak).
Warning in <TCanvas::Constructor>: Deleting canvas with same name: canvas
Info in <TCanvas::Print>: pdf file kinematic_null_Displacement_DCA.pdf has been created
